### Custom model container for GCP
This notebook defines and deploys the trained ML model to GCP. The deployd model is loan default risk classifier.

In [1]:
import os
import joblib
import pickle
from google.cloud import aiplatform

In [2]:
!gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=ouqAlPw6Q1zyQgod7iTBpE2EOKDVXg&access_type=offline&code_challenge=4Yu9CIR_LNk01kvO65ggea6a9c1Cke9Z-Hz-9nOmPew&code_challenge_method=S256


You are now logged in as [jakoslavi@gmail.com].
Your current project is [rovaja-ml-playground].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [3]:
%%writefile requirements.txt
pandas==1.5.3
matplotlib==3.7.1
numpy==1.24.4
scikit-learn==1.3
catboost==1.2.1
joblib==1.3.2
google-cloud-storage>=1.26.0,<2.0.0dev

Overwriting requirements.txt


#### Define GCP values for docker

In [4]:
PROJECT_ID = "rovaja-ml-playground"
REGION = "europe-west1"
BUCKET_URI = "gs://loan_default_app_bucket_euwest"

In [5]:
MODEL_ARTIFACT_DIR = "default_clf_catboost"
D_REPOSITORY = "default-clf-repo"
IMAGE = "default_clf_fastapi_server"
MODEL_DISPLAY_NAME = "default_risk_classifier"

Set project id

In [6]:
!gcloud config set project {PROJECT_ID}

Updated property [core/project].


Uncomment if there is no bucket

In [61]:
#!gsutil mb -l {REGION} {BUCKET_URI}

Creating gs://loan_app_bucket_euwest/...


#### Load customencoders manually to the folder

#### Load model and preprocessing pipelines

In [8]:
model = joblib.load("../default_catboost_clf_1.2.1.joblib")
pipeline = joblib.load("../default_catboost_preprocessor_1.3.0.pkl")

c:\Users\jaakoiv\.conda\envs\datadevs\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 1.3.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\jaakoiv\.conda\envs\datadevs\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator Pipeline from version 1.3.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\jaakoiv\.conda\envs\datadevs\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator SimpleImputer from version 1.3.0 when using version 1.2.2. This might lead to breaking code or inva

In [10]:
%mkdir app

In [11]:
%cd app/
joblib.dump(model, "model.joblib")
with open("preprocessor.pkl", "wb") as f:
    pickle.dump(pipeline, f)

c:\Users\jaakoiv\Documents\GIT_Workspace\Turing_Data_Science_Course\Machine Learning\Capstone\default_catboost_container\app


Push model and preprocessor to GCP

In [12]:
!gsutil cp model.joblib preprocessor.pkl {BUCKET_URI}/{MODEL_ARTIFACT_DIR}/

Copying file://model.joblib [Content-Type=application/octet-stream]...
/ [0 files][    0.0 B/917.0 KiB]                                                
/ [0 files][320.0 KiB/917.0 KiB]                                                
-
- [1 files][917.0 KiB/917.0 KiB]                                                
Copying file://preprocessor.pkl [Content-Type=application/octet-stream]...
- [1 files][917.0 KiB/ 1018 KiB]                                                
\
\ [2 files][ 1018 KiB/ 1018 KiB]                                                

Operation completed over 2 objects/1018.3 KiB.                                   


In [13]:
%cd ..

c:\Users\jaakoiv\Documents\GIT_Workspace\Turing_Data_Science_Course\Machine Learning\Capstone\default_catboost_container


Build FastAPI

In [14]:
%%writefile app/main.py
from fastapi import FastAPI, Request
import joblib
import json
import numpy as np
import pandas as pd
import pickle
import os
import sklearn
from google.cloud import storage
import catboost as cat
from customencoder import FeaturesToSelect
sklearn.set_config(transform_output="pandas")


app = FastAPI()

gcs_client = storage.Client()

with open("preprocessor.pkl", 'wb') as preprocessor_f, open("model.joblib", 'wb') as model_f:
    gcs_client.download_blob_to_file(
        f"{os.environ['AIP_STORAGE_URI']}/preprocessor.pkl", preprocessor_f
    )
    gcs_client.download_blob_to_file(
        f"{os.environ['AIP_STORAGE_URI']}/model.joblib", model_f
    )

with open("preprocessor.pkl", "rb") as f:
    preprocessor = pickle.load(f)

_model = joblib.load("model.joblib")
_preprocessor = preprocessor


@app.get(os.environ['AIP_HEALTH_ROUTE'], status_code=200)
def health():
    return {}


@app.post(os.environ['AIP_PREDICT_ROUTE'])
async def predict(request: Request):
    body = await request.json()
    instances = body["instances"]
    inputs = pd.DataFrame(instances)
    preprocessed_inputs = _preprocessor.transform(inputs)
    preds = _model.predict_proba(preprocessed_inputs)
    preds = preds[:,1]
    
    predictions_list = []
    
    for pred in preds:
        prediction_item = {
            "label": int(pred>0.4),
            "probability": pred
        }
        predictions_list.append(prediction_item)

    predictions_dict = {
        "predictions": predictions_list
    }
    return predictions_dict


Writing app/main.py


Add pre-start script

In [15]:
%%writefile app/prestart.sh
#!/bin/bash
export PORT=$AIP_HTTP_PORT

Writing app/prestart.sh


Create test instances

In [7]:
%%writefile instances.json
{
  "instances": [{"CODE_GENDER_f":0,"FLAG_OWN_CAR_y":0,"FLAG_OWN_REALTY_y":1,"CNT_CHILDREN":0,"AMT_INCOME_TOTAL":202500.0,"AMT_CREDIT":406597.5,"AMT_ANNUITY":24700.5,"AMT_GOODS_PRICE":351000.0,"REGION_POPULATION_RELATIVE":0.018801,"DAYS_BIRTH":-9461,"DAYS_EMPLOYED":-637.0,"DAYS_ID_PUBLISH":-2120,"OWN_CAR_AGE":0,"CNT_FAM_MEMBERS":1.0,"REGION_RATING_CLIENT":1,"REGION_RATING_CLIENT_W_CITY":1,"REG_REGION_NOT_LIVE_REGION_1":0,"REG_REGION_NOT_WORK_REGION_1":0,"LIVE_REGION_NOT_WORK_REGION_1":0,"REG_CITY_NOT_LIVE_CITY_1":0,"REG_CITY_NOT_WORK_CITY_1":0,"LIVE_CITY_NOT_WORK_CITY_1":0,"EXT_SOURCE_1":0.0830369674,"EXT_SOURCE_2":0.2629485927,"EXT_SOURCE_3":0.1393757801,"APARTMENTS_AVG":0.0247,"BASEMENTAREA_AVG":0.0369,"YEARS_BEGINEXPLUATATION_AVG":0.9722,"YEARS_BUILD_AVG":0.6192,"COMMONAREA_AVG":0.0143,"ELEVATORS_AVG":0.0,"ENTRANCES_AVG":0.069,"FLOORSMAX_AVG":0.0833,"FLOORSMIN_AVG":0.125,"LANDAREA_AVG":0.0369,"LIVINGAPARTMENTS_AVG":0.0202,"LIVINGAREA_AVG":0.019,"NONLIVINGAPARTMENTS_AVG":0.0,"NONLIVINGAREA_AVG":0.0,"APARTMENTS_MODE":0.0252,"BASEMENTAREA_MODE":0.0383,"YEARS_BEGINEXPLUATATION_MODE":0.9722,"YEARS_BUILD_MODE":0.6341,"COMMONAREA_MODE":0.0144,"ELEVATORS_MODE":0.0,"ENTRANCES_MODE":0.069,"FLOORSMAX_MODE":0.0833,"FLOORSMIN_MODE":0.125,"LANDAREA_MODE":0.0377,"LIVINGAPARTMENTS_MODE":0.022,"LIVINGAREA_MODE":0.0198,"NONLIVINGAPARTMENTS_MODE":0.0,"NONLIVINGAREA_MODE":0.0,"APARTMENTS_MEDI":0.025,"BASEMENTAREA_MEDI":0.0369,"YEARS_BEGINEXPLUATATION_MEDI":0.9722,"YEARS_BUILD_MEDI":0.6243,"COMMONAREA_MEDI":0.0144,"ELEVATORS_MEDI":0.0,"ENTRANCES_MEDI":0.069,"FLOORSMAX_MEDI":0.0833,"FLOORSMIN_MEDI":0.125,"LANDAREA_MEDI":0.0375,"LIVINGAPARTMENTS_MEDI":0.0205,"LIVINGAREA_MEDI":0.0193,"NONLIVINGAPARTMENTS_MEDI":0.0,"NONLIVINGAREA_MEDI":0.0,"TOTALAREA_MODE":0.0149,"EMERGENCYSTATE_MODE_yes":0.0,"OBS_30_CNT_SOCIAL_CIRCLE":2.0,"DEF_30_CNT_SOCIAL_CIRCLE":2.0,"OBS_60_CNT_SOCIAL_CIRCLE":2.0,"DEF_60_CNT_SOCIAL_CIRCLE":2.0,"DAYS_LAST_PHONE_CHANGE":-1134.0,"FLAG_DOCUMENT_3_1":1,"FLAG_DOCUMENT_5_1":0,"FLAG_DOCUMENT_6_1":0,"FLAG_DOCUMENT_8_1":0,"FLAG_DOCUMENT_9_1":0,"FLAG_DOCUMENT_11_1":0,"FLAG_DOCUMENT_13_1":0,"FLAG_DOCUMENT_14_1":0,"FLAG_DOCUMENT_15_1":0,"FLAG_DOCUMENT_16_1":0,"FLAG_DOCUMENT_18_1":0,"AMT_REQ_CREDIT_BUREAU_HOUR":0.0,"AMT_REQ_CREDIT_BUREAU_DAY":0.0,"AMT_REQ_CREDIT_BUREAU_WEEK":0.0,"AMT_REQ_CREDIT_BUREAU_MON":0.0,"AMT_REQ_CREDIT_BUREAU_QRT":0.0,"AMT_REQ_CREDIT_BUREAU_YEAR":1.0,"HOUSING_INFO_MISSING_RATIO":0.0,"FLAG_CREDIT_BUREAU_MISSING_1":0,"NAME_CONTRACT_TYPE_Cash_loans":1,"NAME_CONTRACT_TYPE_Revolving_loans":0,"NAME_INCOME_TYPE_State_servant":0,"NAME_INCOME_TYPE_Working":1,"NAME_INCOME_TYPE_beneficiary":0,"NAME_INCOME_TYPE_commercial_associate":0,"NAME_EDUCATION_TYPE_Higher_education":0,"NAME_EDUCATION_TYPE_Incomplete_higher":0,"NAME_EDUCATION_TYPE_Lower_secondary":0,"NAME_EDUCATION_TYPE_Secondary___secondary_special":1,"NAME_FAMILY_STATUS_Civil_marriage":0,"NAME_FAMILY_STATUS_Married":0,"NAME_FAMILY_STATUS_Separated":0,"NAME_FAMILY_STATUS_Single___not_married":1,"NAME_FAMILY_STATUS_Widow":0,"NAME_HOUSING_TYPE_Co_op_apartment":0,"NAME_HOUSING_TYPE_House___apartment":1,"NAME_HOUSING_TYPE_Municipal_apartment":0,"NAME_HOUSING_TYPE_Office_apartment":0,"NAME_HOUSING_TYPE_Rented_apartment":0,"NAME_HOUSING_TYPE_With_parents":0,"OCCUPATION_TYPE_Accountants":0,"OCCUPATION_TYPE_Cooking_staff":0,"OCCUPATION_TYPE_Core_staff":0,"OCCUPATION_TYPE_Drivers":0,"OCCUPATION_TYPE_High_skill_tech_staff":0,"OCCUPATION_TYPE_Laborers":1,"OCCUPATION_TYPE_Managers":0,"OCCUPATION_TYPE_Medicine_staff":0,"OCCUPATION_TYPE_Private_service_staff":0,"OCCUPATION_TYPE_Security_staff":0,"OCCUPATION_TYPE_administration_staff":0,"OCCUPATION_TYPE_low_skill_Laborers":0,"OCCUPATION_TYPE_sales_staff":0,"ORGANIZATION_TYPE_Business_Entity":1,"ORGANIZATION_TYPE_Construction":0,"ORGANIZATION_TYPE_Finance":0,"ORGANIZATION_TYPE_House":0,"ORGANIZATION_TYPE_Industry":0,"ORGANIZATION_TYPE_Other":0,"ORGANIZATION_TYPE_Public":0,"ORGANIZATION_TYPE_Self_employed":0,"ORGANIZATION_TYPE_Services":0,"ORGANIZATION_TYPE_Trade":0,"ORGANIZATION_TYPE_Transport":0,"FONDKAPREMONT_MODE_not_specified":0,"FONDKAPREMONT_MODE_org_spec_account":0,"FONDKAPREMONT_MODE_reg_oper_account":1,"FONDKAPREMONT_MODE_reg_oper_spec_account":0,"HOUSETYPE_MODE_block_of_flats":1,"HOUSETYPE_MODE_specific_housing":0,"HOUSETYPE_MODE_terraced_house":0,"WALLSMATERIAL_MODE_Block":0,"WALLSMATERIAL_MODE_Mixed":0,"WALLSMATERIAL_MODE_Monolithic":0,"WALLSMATERIAL_MODE_Others":0,"WALLSMATERIAL_MODE_Panel":0,"WALLSMATERIAL_MODE_Stone__brick":1,"WALLSMATERIAL_MODE_Wooden":0,"PREV_AMT_ANNUITY_MEAN":0.0000000006,"PREV_AMT_ANNUITY_MAX":0.0000000006,"PREV_AMT_APPLICATION_MEAN":0.0000000124,"PREV_AMT_APPLICATION_MAX":0.0000000124,"PREV_AMT_CREDIT_MEAN":0.0000000124,"PREV_AMT_CREDIT_MAX":0.0000000124,"PREV_AMT_DOWN_PAYMENT_MEAN":0.0,"PREV_AMT_DOWN_PAYMENT_MAX":0.0,"PREV_AMT_GOODS_PRICE_MEAN":0.0000000124,"PREV_AMT_GOODS_PRICE_MAX":0.0000000124,"PREV_RATE_DOWN_PAYMENT_MEAN":0.0,"PREV_RATE_DOWN_PAYMENT_MAX":0.0,"PREV_DAYS_DECISION_MEAN":-0.0,"PREV_DAYS_DECISION_MAX":-0.0,"PREV_SELLERPLACE_AREA_MEAN":0.0,"PREV_SELLERPLACE_AREA_MAX":0.0,"PREV_CNT_PAYMENT_MEAN":0.0,"PREV_CNT_PAYMENT_MAX":0.0,"PREV_DAYS_FIRST_DRAWING_MEAN":0,"PREV_DAYS_FIRST_DRAWING_MAX":0,"PREV_DAYS_FIRST_DUE_MEAN":-0.0,"PREV_DAYS_FIRST_DUE_MAX":-0.0,"PREV_DAYS_LAST_DUE_1ST_VERSION_MEAN":0.0,"PREV_DAYS_LAST_DUE_1ST_VERSION_MAX":0.0,"PREV_DAYS_LAST_DUE_MEAN":-0.0,"PREV_DAYS_LAST_DUE_MAX":-0.0,"PREV_DAYS_TERMINATION_MEAN":-0.0,"PREV_DAYS_TERMINATION_MAX":-0.0,"PREV_FLAG_MISSING_ANNUITY_MEAN":0.0,"PREV_FLAG_MISSING_ANNUITY_MAX":0.0,"PREV_INTEREST_AMT_MEAN":0.000000003,"PREV_INTEREST_AMT_MAX":0.000000003,"PREV_INTEREST_RATIO_MEAN":0.0,"PREV_INTEREST_RATIO_MAX":0.0,"PREV_INTEREST_RATE_MEAN":0.0,"PREV_INTEREST_RATE_MAX":0.0,"PREV_AMT_REFUSED_MEAN":0.0,"PREV_AMT_REFUSED_MAX":0.0,"PREV_AMT_REFUSED_RATIO_MEAN":0.0,"PREV_AMT_REFUSED_RATIO_MAX":0.0,"PREV_AMT_CREDIT_GOODS_RATIO_MEAN":0.0,"PREV_AMT_CREDIT_GOODS_RATIO_MAX":0.0,"PREV_MISSING_VALUES_RATIO_MEAN":0.0,"PREV_MISSING_VALUES_RATIO_MAX":0.0,"PREV_CREDIT_DOWNPAYMENT_RATIO_MEAN":0.0,"PREV_CREDIT_DOWNPAYMENT_RATIO_MAX":0.0,"PREV_NAME_CONTRACT_TYPE_Cash_loans_MEAN":0.0,"PREV_NAME_CONTRACT_TYPE_Cash_loans_SUM":0.0,"PREV_NAME_CONTRACT_TYPE_Consumer_loans_MEAN":0.0,"PREV_NAME_CONTRACT_TYPE_Consumer_loans_SUM":0.0,"PREV_NAME_CONTRACT_TYPE_Revolving_loans_MEAN":0.0,"PREV_NAME_CONTRACT_TYPE_Revolving_loans_SUM":0.0,"PREV_NAME_CASH_LOAN_PURPOSE_Other_MEAN":0.0,"PREV_NAME_CASH_LOAN_PURPOSE_Other_SUM":0.0,"PREV_NAME_CASH_LOAN_PURPOSE_Repairs_MEAN":0.0,"PREV_NAME_CASH_LOAN_PURPOSE_Repairs_SUM":0.0,"PREV_NAME_CASH_LOAN_PURPOSE_Urgent_needs_MEAN":0.0,"PREV_NAME_CASH_LOAN_PURPOSE_Urgent_needs_SUM":0.0,"PREV_NAME_CASH_LOAN_PURPOSE_XAP_MEAN":0.0,"PREV_NAME_CASH_LOAN_PURPOSE_XAP_SUM":0.0,"PREV_NAME_PAYMENT_TYPE_Cash_through_the_bank_MEAN":0.0,"PREV_NAME_PAYMENT_TYPE_Cash_through_the_bank_SUM":0.0,"PREV_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_MEAN":0.0,"PREV_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_SUM":0.0,"PREV_NAME_PAYMENT_TYPE_Non_cash_from_your_account_MEAN":0.0,"PREV_NAME_PAYMENT_TYPE_Non_cash_from_your_account_SUM":0.0,"PREV_CODE_REJECT_REASON_CLIENT_MEAN":0.0,"PREV_CODE_REJECT_REASON_CLIENT_SUM":0.0,"PREV_CODE_REJECT_REASON_HC_MEAN":0.0,"PREV_CODE_REJECT_REASON_HC_SUM":0.0,"PREV_CODE_REJECT_REASON_LIMIT_MEAN":0.0,"PREV_CODE_REJECT_REASON_LIMIT_SUM":0.0,"PREV_CODE_REJECT_REASON_SCO_MEAN":0.0,"PREV_CODE_REJECT_REASON_SCO_SUM":0.0,"PREV_CODE_REJECT_REASON_SCOFR_MEAN":0.0,"PREV_CODE_REJECT_REASON_SCOFR_SUM":0.0,"PREV_CODE_REJECT_REASON_SYSTEM_MEAN":0.0,"PREV_CODE_REJECT_REASON_SYSTEM_SUM":0.0,"PREV_CODE_REJECT_REASON_VERIF_MEAN":0.0,"PREV_CODE_REJECT_REASON_VERIF_SUM":0.0,"PREV_CODE_REJECT_REASON_XAP_MEAN":0.0,"PREV_CODE_REJECT_REASON_XAP_SUM":0.0,"PREV_NAME_GOODS_CATEGORY_Audio_Video_MEAN":0.0,"PREV_NAME_GOODS_CATEGORY_Audio_Video_SUM":0.0,"PREV_NAME_GOODS_CATEGORY_Auto_Accessories_MEAN":0.0,"PREV_NAME_GOODS_CATEGORY_Auto_Accessories_SUM":0.0,"PREV_NAME_GOODS_CATEGORY_Clothing_and_Accessories_MEAN":0.0,"PREV_NAME_GOODS_CATEGORY_Clothing_and_Accessories_SUM":0.0,"PREV_NAME_GOODS_CATEGORY_Computers_MEAN":0.0,"PREV_NAME_GOODS_CATEGORY_Computers_SUM":0.0,"PREV_NAME_GOODS_CATEGORY_Construction_Materials_MEAN":0.0,"PREV_NAME_GOODS_CATEGORY_Construction_Materials_SUM":0.0,"PREV_NAME_GOODS_CATEGORY_Consumer_Electronics_MEAN":0.0,"PREV_NAME_GOODS_CATEGORY_Consumer_Electronics_SUM":0.0,"PREV_NAME_GOODS_CATEGORY_Furniture_MEAN":0.0,"PREV_NAME_GOODS_CATEGORY_Furniture_SUM":0.0,"PREV_NAME_GOODS_CATEGORY_Gardening_MEAN":0.0,"PREV_NAME_GOODS_CATEGORY_Gardening_SUM":0.0,"PREV_NAME_GOODS_CATEGORY_Homewares_MEAN":0.0,"PREV_NAME_GOODS_CATEGORY_Homewares_SUM":0.0,"PREV_NAME_GOODS_CATEGORY_Jewelry_MEAN":0.0,"PREV_NAME_GOODS_CATEGORY_Jewelry_SUM":0.0,"PREV_NAME_GOODS_CATEGORY_Medical_Supplies_MEAN":0.0,"PREV_NAME_GOODS_CATEGORY_Medical_Supplies_SUM":0.0,"PREV_NAME_GOODS_CATEGORY_Medicine_MEAN":0.0,"PREV_NAME_GOODS_CATEGORY_Medicine_SUM":0.0,"PREV_NAME_GOODS_CATEGORY_Mobile_MEAN":0.0,"PREV_NAME_GOODS_CATEGORY_Mobile_SUM":0.0,"PREV_NAME_GOODS_CATEGORY_Office_Appliances_MEAN":0.0,"PREV_NAME_GOODS_CATEGORY_Office_Appliances_SUM":0.0,"PREV_NAME_GOODS_CATEGORY_Other_MEAN":0.0,"PREV_NAME_GOODS_CATEGORY_Other_SUM":0.0,"PREV_NAME_GOODS_CATEGORY_Photo___Cinema_Equipment_MEAN":0.0,"PREV_NAME_GOODS_CATEGORY_Photo___Cinema_Equipment_SUM":0.0,"PREV_NAME_GOODS_CATEGORY_Sport_and_Leisure_MEAN":0.0,"PREV_NAME_GOODS_CATEGORY_Sport_and_Leisure_SUM":0.0,"PREV_NAME_GOODS_CATEGORY_Tourism_MEAN":0.0,"PREV_NAME_GOODS_CATEGORY_Tourism_SUM":0.0,"PREV_NAME_GOODS_CATEGORY_Vehicles_MEAN":0.0,"PREV_NAME_GOODS_CATEGORY_Vehicles_SUM":0.0,"PREV_NAME_PORTFOLIO_Cards_MEAN":0.0,"PREV_NAME_PORTFOLIO_Cards_SUM":0.0,"PREV_NAME_PORTFOLIO_Cars_MEAN":0.0,"PREV_NAME_PORTFOLIO_Cars_SUM":0.0,"PREV_NAME_PORTFOLIO_Cash_MEAN":0.0,"PREV_NAME_PORTFOLIO_Cash_SUM":0.0,"PREV_NAME_PORTFOLIO_POS_MEAN":0.0,"PREV_NAME_PORTFOLIO_POS_SUM":0.0,"PREV_CHANNEL_TYPE_AP___Cash_loan__MEAN":0.0,"PREV_CHANNEL_TYPE_AP___Cash_loan__SUM":0.0,"PREV_CHANNEL_TYPE_Car_dealer_MEAN":0.0,"PREV_CHANNEL_TYPE_Car_dealer_SUM":0.0,"PREV_CHANNEL_TYPE_Contact_center_MEAN":0.0,"PREV_CHANNEL_TYPE_Contact_center_SUM":0.0,"PREV_CHANNEL_TYPE_Country_wide_MEAN":0.0,"PREV_CHANNEL_TYPE_Country_wide_SUM":0.0,"PREV_CHANNEL_TYPE_Credit_and_cash_offices_MEAN":0.0,"PREV_CHANNEL_TYPE_Credit_and_cash_offices_SUM":0.0,"PREV_CHANNEL_TYPE_Regional___Local_MEAN":0.0,"PREV_CHANNEL_TYPE_Regional___Local_SUM":0.0,"PREV_CHANNEL_TYPE_Stone_MEAN":0.0,"PREV_CHANNEL_TYPE_Stone_SUM":0.0,"PREV_NAME_SELLER_INDUSTRY_Auto_technology_MEAN":0.0,"PREV_NAME_SELLER_INDUSTRY_Auto_technology_SUM":0.0,"PREV_NAME_SELLER_INDUSTRY_Clothing_MEAN":0.0,"PREV_NAME_SELLER_INDUSTRY_Clothing_SUM":0.0,"PREV_NAME_SELLER_INDUSTRY_Connectivity_MEAN":0.0,"PREV_NAME_SELLER_INDUSTRY_Connectivity_SUM":0.0,"PREV_NAME_SELLER_INDUSTRY_Construction_MEAN":0.0,"PREV_NAME_SELLER_INDUSTRY_Construction_SUM":0.0,"PREV_NAME_SELLER_INDUSTRY_Consumer_electronics_MEAN":0.0,"PREV_NAME_SELLER_INDUSTRY_Consumer_electronics_SUM":0.0,"PREV_NAME_SELLER_INDUSTRY_Furniture_MEAN":0.0,"PREV_NAME_SELLER_INDUSTRY_Furniture_SUM":0.0,"PREV_NAME_SELLER_INDUSTRY_Industry_MEAN":0.0,"PREV_NAME_SELLER_INDUSTRY_Industry_SUM":0.0,"PREV_NAME_SELLER_INDUSTRY_Jewelry_MEAN":0.0,"PREV_NAME_SELLER_INDUSTRY_Jewelry_SUM":0.0,"PREV_NAME_SELLER_INDUSTRY_MLM_partners_MEAN":0.0,"PREV_NAME_SELLER_INDUSTRY_MLM_partners_SUM":0.0,"PREV_NAME_SELLER_INDUSTRY_Tourism_MEAN":0.0,"PREV_NAME_SELLER_INDUSTRY_Tourism_SUM":0.0,"PREV_PRODUCT_COMBINATION_Card_Street_MEAN":0.0,"PREV_PRODUCT_COMBINATION_Card_Street_SUM":0.0,"PREV_PRODUCT_COMBINATION_Card_X_Sell_MEAN":0.0,"PREV_PRODUCT_COMBINATION_Card_X_Sell_SUM":0.0,"PREV_PRODUCT_COMBINATION_Cash_MEAN":0.0,"PREV_PRODUCT_COMBINATION_Cash_SUM":0.0,"PREV_PRODUCT_COMBINATION_Cash_Street__high_MEAN":0.0,"PREV_PRODUCT_COMBINATION_Cash_Street__high_SUM":0.0,"PREV_PRODUCT_COMBINATION_Cash_Street__low_MEAN":0.0,"PREV_PRODUCT_COMBINATION_Cash_Street__low_SUM":0.0,"PREV_PRODUCT_COMBINATION_Cash_Street__middle_MEAN":0.0,"PREV_PRODUCT_COMBINATION_Cash_Street__middle_SUM":0.0,"PREV_PRODUCT_COMBINATION_Cash_X_Sell__high_MEAN":0.0,"PREV_PRODUCT_COMBINATION_Cash_X_Sell__high_SUM":0.0,"PREV_PRODUCT_COMBINATION_Cash_X_Sell__low_MEAN":0.0,"PREV_PRODUCT_COMBINATION_Cash_X_Sell__low_SUM":0.0,"PREV_PRODUCT_COMBINATION_Cash_X_Sell__middle_MEAN":0.0,"PREV_PRODUCT_COMBINATION_Cash_X_Sell__middle_SUM":0.0,"PREV_PRODUCT_COMBINATION_POS_household_with_interest_MEAN":0.0,"PREV_PRODUCT_COMBINATION_POS_household_with_interest_SUM":0.0,"PREV_PRODUCT_COMBINATION_POS_household_without_interest_MEAN":0.0,"PREV_PRODUCT_COMBINATION_POS_household_without_interest_SUM":0.0,"PREV_PRODUCT_COMBINATION_POS_industry_with_interest_MEAN":0.0,"PREV_PRODUCT_COMBINATION_POS_industry_with_interest_SUM":0.0,"PREV_PRODUCT_COMBINATION_POS_industry_without_interest_MEAN":0.0,"PREV_PRODUCT_COMBINATION_POS_industry_without_interest_SUM":0.0,"PREV_PRODUCT_COMBINATION_POS_mobile_with_interest_MEAN":0.0,"PREV_PRODUCT_COMBINATION_POS_mobile_with_interest_SUM":0.0,"PREV_PRODUCT_COMBINATION_POS_mobile_without_interest_MEAN":0.0,"PREV_PRODUCT_COMBINATION_POS_mobile_without_interest_SUM":0.0,"PREV_PRODUCT_COMBINATION_POS_other_with_interest_MEAN":0.0,"PREV_PRODUCT_COMBINATION_POS_other_with_interest_SUM":0.0,"PREV_PRODUCT_COMBINATION_POS_others_without_interest_MEAN":0.0,"PREV_PRODUCT_COMBINATION_POS_others_without_interest_SUM":0.0,"PREV_NAME_CONTRACT_STATUS_MEAN":0.0,"PREV_NAME_CONTRACT_STATUS_SUM":0.0,"PREV_NAME_CLIENT_TYPE_MEAN":0.0,"PREV_NAME_CLIENT_TYPE_SUM":0.0,"PREV_NAME_PRODUCT_TYPE_x_sell_MEAN":0,"PREV_NAME_PRODUCT_TYPE_x_sell_SUM":0.0,"PREV_NAME_YIELD_GROUP_MEAN":0.0,"PREV_NAME_YIELD_GROUP_SUM":0.0,"PREV_FLAG_LAST_APPL_PER_CONTRACT_y_MEAN":0.0,"PREV_FLAG_LAST_APPL_PER_CONTRACT_y_SUM":0.0,"PREV_NFLAG_LAST_APPL_IN_DAY_1_MEAN":0.0,"PREV_NFLAG_LAST_APPL_IN_DAY_1_SUM":0.0,"PREV_NFLAG_INSURED_ON_APPROVAL_1_0_MEAN":0.0,"PREV_NFLAG_INSURED_ON_APPROVAL_1_0_SUM":0.0,"PREV_FLAG_MISSING_FIRST_DRAW_1_MEAN":0.0,"PREV_FLAG_MISSING_FIRST_DRAW_1_SUM":0.0,"PREV_REFUSED_APPLICATION_1_MEAN":0.0,"PREV_REFUSED_APPLICATION_1_SUM":0.0,"APPROVED_AMT_ANNUITY_MEAN":9251.775,"APPROVED_AMT_ANNUITY_MAX":9251.775,"APPROVED_AMT_APPLICATION_MEAN":179055.0,"APPROVED_AMT_APPLICATION_MAX":179055.0,"APPROVED_AMT_CREDIT_MEAN":179055.0,"APPROVED_AMT_CREDIT_MAX":179055.0,"APPROVED_AMT_DOWN_PAYMENT_MEAN":0.0,"APPROVED_AMT_DOWN_PAYMENT_MAX":0.0,"APPROVED_AMT_GOODS_PRICE_MEAN":179055.0,"APPROVED_AMT_GOODS_PRICE_MAX":179055.0,"APPROVED_RATE_DOWN_PAYMENT_MEAN":0.0,"APPROVED_RATE_DOWN_PAYMENT_MAX":0.0,"APPROVED_DAYS_DECISION_MEAN":-606.0,"APPROVED_DAYS_DECISION_MAX":-606.0,"APPROVED_SELLERPLACE_AREA_MEAN":500.0,"APPROVED_SELLERPLACE_AREA_MAX":500.0,"APPROVED_CNT_PAYMENT_MEAN":24.0,"APPROVED_CNT_PAYMENT_MAX":24.0,"APPROVED_DAYS_FIRST_DRAWING_MEAN":0,"APPROVED_DAYS_FIRST_DRAWING_MAX":0,"APPROVED_DAYS_FIRST_DUE_MEAN":-565.0,"APPROVED_DAYS_FIRST_DUE_MAX":-565.0,"APPROVED_DAYS_LAST_DUE_1ST_VERSION_MEAN":125.0,"APPROVED_DAYS_LAST_DUE_1ST_VERSION_MAX":125.0,"APPROVED_DAYS_LAST_DUE_MEAN":-25.0,"APPROVED_DAYS_LAST_DUE_MAX":-25.0,"APPROVED_DAYS_TERMINATION_MEAN":-17.0,"APPROVED_DAYS_TERMINATION_MAX":-17.0,"APPROVED_FLAG_MISSING_ANNUITY_MEAN":0.0,"APPROVED_FLAG_MISSING_ANNUITY_MAX":0.0,"APPROVED_INTEREST_AMT_MEAN":42987.6,"APPROVED_INTEREST_AMT_MAX":42987.6,"APPROVED_INTEREST_RATIO_MEAN":0.2400804222,"APPROVED_INTEREST_RATIO_MAX":0.2400804222,"APPROVED_INTEREST_RATE_MEAN":0.2304772053,"APPROVED_INTEREST_RATE_MAX":0.2304772053,"APPROVED_AMT_REFUSED_MEAN":0.0,"APPROVED_AMT_REFUSED_MAX":0.0,"APPROVED_AMT_REFUSED_RATIO_MEAN":1.0,"APPROVED_AMT_REFUSED_RATIO_MAX":1.0,"APPROVED_AMT_CREDIT_GOODS_RATIO_MEAN":1.0,"APPROVED_AMT_CREDIT_GOODS_RATIO_MAX":1.0,"APPROVED_MISSING_VALUES_RATIO_MEAN":0.075,"APPROVED_MISSING_VALUES_RATIO_MAX":0.075,"APPROVED_CREDIT_DOWNPAYMENT_RATIO_MEAN":0.0,"APPROVED_CREDIT_DOWNPAYMENT_RATIO_MAX":0.0,"INSTAL_NUM_INSTALMENT_VERSION_NUNIQUE":19.0,"INSTAL_DAYS_INSTALMENT_MAX":-0.0000000003,"INSTAL_DAYS_INSTALMENT_MEAN":-0.6525031716,"INSTAL_DAYS_ENTRY_PAYMENT_MAX":-0.0000000003,"INSTAL_DAYS_ENTRY_PAYMENT_MEAN":-1.0690681524,"INSTAL_AMT_INSTALMENT_MAX":15211.6126257717,"INSTAL_AMT_INSTALMENT_MEAN":840.680640236,"INSTAL_AMT_PAYMENT_MAX":15211.6126257717,"INSTAL_AMT_PAYMENT_MEAN":840.680640236,"INSTAL_DAYS_PASS_DUE_MAX":0.0,"INSTAL_DAYS_PASS_DUE_MEAN":0.0,"INSTAL_DAYS_PASS_DUE_SUM":0.0,"INSTAL_DAYS_BEFORE_DUE_MAX":6.8761151246,"INSTAL_DAYS_BEFORE_DUE_MEAN":0.4165649809,"INSTAL_DAYS_BEFORE_DUE_SUM":7.9147346363,"INSTAL_PAYMENT_INST_RATIO_MAX":0.2865047969,"INSTAL_PAYMENT_INST_RATIO_MEAN":0.0194102011,"INSTAL_INST_PAYMENT_DIFFERENCE_MAX":0.0,"INSTAL_INST_PAYMENT_DIFFERENCE_MEAN":0.0,"INSTAL_COUNT":19.0,"CC_MONTHS_BALANCE_MAX":0,"CC_MONTHS_BALANCE_MEAN":0,"CC_AMT_BALANCE_MAX":0,"CC_AMT_BALANCE_MEAN":0,"CC_AMT_CREDIT_LIMIT_ACTUAL_MAX":0,"CC_AMT_CREDIT_LIMIT_ACTUAL_MEAN":0,"CC_AMT_DRAWINGS_ATM_CURRENT_MAX":0,"CC_AMT_DRAWINGS_ATM_CURRENT_MEAN":0,"CC_AMT_DRAWINGS_CURRENT_MAX":0,"CC_AMT_DRAWINGS_CURRENT_MEAN":0,"CC_AMT_DRAWINGS_OTHER_CURRENT_MAX":0,"CC_AMT_DRAWINGS_OTHER_CURRENT_MEAN":0,"CC_AMT_DRAWINGS_POS_CURRENT_MAX":0,"CC_AMT_DRAWINGS_POS_CURRENT_MEAN":0,"CC_AMT_INST_MIN_REGULARITY_MAX":0,"CC_AMT_INST_MIN_REGULARITY_MEAN":0,"CC_AMT_PAYMENT_CURRENT_MAX":0,"CC_AMT_PAYMENT_CURRENT_MEAN":0,"CC_AMT_PAYMENT_TOTAL_CURRENT_MAX":0,"CC_AMT_PAYMENT_TOTAL_CURRENT_MEAN":0,"CC_AMT_RECEIVABLE_PRINCIPAL_MAX":0,"CC_AMT_RECEIVABLE_PRINCIPAL_MEAN":0,"CC_AMT_RECIVABLE_MAX":0,"CC_AMT_RECIVABLE_MEAN":0,"CC_AMT_TOTAL_RECEIVABLE_MAX":0,"CC_AMT_TOTAL_RECEIVABLE_MEAN":0,"CC_CNT_DRAWINGS_ATM_CURRENT_MEAN":0,"CC_CNT_DRAWINGS_ATM_CURRENT_SUM":0,"CC_CNT_DRAWINGS_CURRENT_MEAN":0,"CC_CNT_DRAWINGS_CURRENT_SUM":0,"CC_CNT_DRAWINGS_OTHER_CURRENT_MEAN":0,"CC_CNT_DRAWINGS_OTHER_CURRENT_SUM":0,"CC_CNT_DRAWINGS_POS_CURRENT_MEAN":0,"CC_CNT_DRAWINGS_POS_CURRENT_SUM":0,"CC_CNT_INSTALMENT_MATURE_CUM_MEAN":0,"CC_CNT_INSTALMENT_MATURE_CUM_SUM":0,"CC_SK_DPD_MAX":0,"CC_SK_DPD_MEAN":0,"CC_SK_DPD_DEF_MAX":0,"CC_SK_DPD_DEF_MEAN":0,"CC_NAME_CONTRACT_STATUS_Active_MEAN":0,"CC_NAME_CONTRACT_STATUS_Active_SUM":0,"CC_NAME_CONTRACT_STATUS_Approved_MEAN":0,"CC_NAME_CONTRACT_STATUS_Approved_SUM":0,"CC_NAME_CONTRACT_STATUS_Completed_MEAN":0,"CC_NAME_CONTRACT_STATUS_Completed_SUM":0,"CC_NAME_CONTRACT_STATUS_Demand_MEAN":0,"CC_NAME_CONTRACT_STATUS_Demand_SUM":0,"CC_NAME_CONTRACT_STATUS_Refused_MEAN":0,"CC_NAME_CONTRACT_STATUS_Refused_SUM":0,"CC_NAME_CONTRACT_STATUS_Sent_proposal_MEAN":0,"CC_NAME_CONTRACT_STATUS_Sent_proposal_SUM":0,"CC_NAME_CONTRACT_STATUS_Signed_MEAN":0,"CC_NAME_CONTRACT_STATUS_Signed_SUM":0,"CC_FLAG_MISSING_INST_1_MEAN":0,"CC_FLAG_MISSING_INST_1_SUM":0,"CC_FLAG_MISSING_DRAWING_1_MEAN":0,"CC_FLAG_MISSING_DRAWING_1_SUM":0,"CC_COUNT":0,"POS_MONTHS_BALANCE_MAX":-0.9512294245,"POS_MONTHS_BALANCE_MEAN":-5.3649645884,"POS_CNT_INSTALMENT_MEAN":15.1087644403,"POS_CNT_INSTALMENT_SUM":287.0665243659,"POS_CNT_INSTALMENT_FUTURE_MEAN":8.5126238468,"POS_CNT_INSTALMENT_FUTURE_SUM":161.7398530886,"POS_SK_DPD_MAX":0.0,"POS_SK_DPD_MEAN":0.0,"POS_SK_DPD_DEF_MAX":0.0,"POS_SK_DPD_DEF_MEAN":0.0,"POS_NAME_CONTRACT_STATUS_Active_MEAN":0.6295318517,"POS_NAME_CONTRACT_STATUS_Active_SUM":11.9611051819,"POS_NAME_CONTRACT_STATUS_Amortized_debt_MEAN":0.0,"POS_NAME_CONTRACT_STATUS_Amortized_debt_SUM":0.0,"POS_NAME_CONTRACT_STATUS_Approved_MEAN":0.0,"POS_NAME_CONTRACT_STATUS_Approved_SUM":0.0,"POS_NAME_CONTRACT_STATUS_Canceled_MEAN":0.0,"POS_NAME_CONTRACT_STATUS_Canceled_SUM":0.0,"POS_NAME_CONTRACT_STATUS_Completed_MEAN":0.0,"POS_NAME_CONTRACT_STATUS_Completed_SUM":0.0,"POS_NAME_CONTRACT_STATUS_Demand_MEAN":0.0,"POS_NAME_CONTRACT_STATUS_Demand_SUM":0.0,"POS_NAME_CONTRACT_STATUS_Returned_to_the_store_MEAN":0.0,"POS_NAME_CONTRACT_STATUS_Returned_to_the_store_SUM":0.0,"POS_NAME_CONTRACT_STATUS_Signed_MEAN":0.0,"POS_NAME_CONTRACT_STATUS_Signed_SUM":0.0,"POS_NAME_CONTRACT_STATUS_XNA_MEAN":0.0,"POS_NAME_CONTRACT_STATUS_XNA_SUM":0.0,"POS_COUNT":19.0,"BURO_DAYS_CREDIT_MEAN":-0.0746673386,"BURO_DAYS_CREDIT_MAX":-8.982521045e-29,"BURO_CREDIT_DAY_OVERDUE_MEAN":0.0,"BURO_CREDIT_DAY_OVERDUE_MAX":0.0,"BURO_DAYS_CREDIT_ENDDATE_MEAN":0.0,"BURO_DAYS_CREDIT_ENDDATE_MAX":0.0,"BURO_DAYS_ENDDATE_FACT_MEAN":-0.0000000004,"BURO_DAYS_ENDDATE_FACT_MAX":-7.407298148e-29,"BURO_AMT_CREDIT_MAX_OVERDUE_MEAN":0.0469751783,"BURO_AMT_CREDIT_MAX_OVERDUE_MAX":0.2348758914,"BURO_CNT_CREDIT_PROLONG_SUM":0.0,"BURO_AMT_CREDIT_SUM_MEAN":23.1893293834,"BURO_AMT_CREDIT_SUM_MAX":185.5146350659,"BURO_AMT_CREDIT_SUM_DEBT_MEAN":1.161009559e-18,"BURO_AMT_CREDIT_SUM_DEBT_MAX":5.805047794e-18,"BURO_AMT_CREDIT_SUM_LIMIT_MEAN":46.3786587665,"BURO_AMT_CREDIT_SUM_LIMIT_MAX":185.5146350659,"BURO_AMT_CREDIT_SUM_OVERDUE_MEAN":0.0,"BURO_AMT_CREDIT_SUM_OVERDUE_MAX":0.0,"BURO_DAYS_CREDIT_UPDATE_MEAN":-0.0173982145,"BURO_DAYS_CREDIT_UPDATE_MAX":-7.407298148e-29,"BURO_AMT_ANNUITY_MEAN":0.0,"BURO_AMT_ANNUITY_MAX":0.0,"BURO_FLAG_MISSING_AMT_ANNUITY_MEAN":0.0,"BURO_FLAG_MISSING_AMT_ANNUITY_MAX":0.0,"BURO_FLAG_MISSING_CREDIT_MAX_OVERDUE_MEAN":0.0,"BURO_FLAG_MISSING_CREDIT_MAX_OVERDUE_MAX":0.0,"BURO_CREDIT_INFO_MISSING_RATIO_MEAN":0.0003624628,"BURO_CREDIT_INFO_MISSING_RATIO_MAX":0.0028997024,"BURO_AMT_CREDIT_DEBT_RATE_MEAN":2.580015509e-24,"BURO_AMT_CREDIT_DEBT_RATE_MAX":1.290007754e-23,"BURO_FLAG_ACTIVE_CREDIT_MEAN":0.0007249256,"BURO_FLAG_ACTIVE_CREDIT_MAX":0.0057994047,"BURO_MONTHS_BALANCE_MIN_MIN":-0.0173982142,"BURO_MONTHS_BALANCE_MAX_MAX":0.0,"BURO_MONTHS_BALANCE_SIZE_MEAN":0.0028997025,"BURO_MONTHS_BALANCE_SIZE_MAX":0.0231976189,"BURO_CREDIT_TYPE_Car_loan_MEAN":0.0,"BURO_CREDIT_TYPE_Car_loan_SUM":0.0,"BURO_CREDIT_TYPE_Consumer_credit_MEAN":0.0,"BURO_CREDIT_TYPE_Consumer_credit_SUM":0.0,"BURO_CREDIT_TYPE_Credit_card_MEAN":0.0007249256,"BURO_CREDIT_TYPE_Credit_card_SUM":0.0057994048,"BURO_CREDIT_TYPE_Loan_for_business_development_MEAN":0.0,"BURO_CREDIT_TYPE_Loan_for_business_development_SUM":0.0,"BURO_CREDIT_TYPE_Microloan_MEAN":0.0,"BURO_CREDIT_TYPE_Microloan_SUM":0.0,"BURO_CREDIT_TYPE_Mortgage_MEAN":0.0,"BURO_CREDIT_TYPE_Mortgage_SUM":0.0,"BURO_CREDIT_TYPE_Other_MEAN":0.0,"BURO_CREDIT_TYPE_Other_SUM":0.0,"BURO_CREDIT_CURRENCY_currency_1_MEAN":0.0007249256,"BURO_CREDIT_CURRENCY_currency_1_SUM":0.0057994048,"BURO_CREDIT_CURRENCY_currency_2_MEAN":0.0,"BURO_CREDIT_CURRENCY_currency_2_SUM":0.0,"BURO_CREDIT_CURRENCY_currency_3_MEAN":0.0,"BURO_CREDIT_CURRENCY_currency_3_SUM":0.0,"BURO_CREDIT_CURRENCY_currency_4_MEAN":0.0,"BURO_CREDIT_CURRENCY_currency_4_SUM":0.0,"BURO_CREDIT_ACTIVE_Active_MEAN":0.0007249256,"BURO_CREDIT_ACTIVE_Active_SUM":0.0057994047,"BURO_CREDIT_ACTIVE_Bad_debt_MEAN":0.0,"BURO_CREDIT_ACTIVE_Bad_debt_SUM":0.0,"BURO_CREDIT_ACTIVE_Closed_MEAN":0.0,"BURO_CREDIT_ACTIVE_Closed_SUM":0.0,"BURO_CREDIT_ACTIVE_Sold_MEAN":0.0,"BURO_CREDIT_ACTIVE_Sold_SUM":0.0,"BURO_STATUS_0_MEAN_MEAN":0.0003624628,"BURO_STATUS_0_MEAN_SUM":0.0028997024,"BURO_STATUS_0_SUM_MEAN":0.0014498512,"BURO_STATUS_0_SUM_SUM":0.0115988096,"BURO_STATUS_1_MEAN_MEAN":6.164288801e-16,"BURO_STATUS_1_MEAN_SUM":0.0,"BURO_STATUS_1_SUM_MEAN":0.0,"BURO_STATUS_1_SUM_SUM":0.0,"BURO_STATUS_2_MEAN_MEAN":0.0,"BURO_STATUS_2_MEAN_SUM":0.0,"BURO_STATUS_2_SUM_MEAN":0.0,"BURO_STATUS_2_SUM_SUM":0.0,"BURO_STATUS_3_MEAN_MEAN":0.0,"BURO_STATUS_3_MEAN_SUM":0.0,"BURO_STATUS_3_SUM_MEAN":0.0,"BURO_STATUS_3_SUM_SUM":0.0,"BURO_STATUS_4_MEAN_MEAN":0.0,"BURO_STATUS_4_MEAN_SUM":0.0,"BURO_STATUS_4_SUM_MEAN":0.0,"BURO_STATUS_4_SUM_SUM":0.0,"BURO_STATUS_5_MEAN_MEAN":0.0,"BURO_STATUS_5_MEAN_SUM":0.0,"BURO_STATUS_5_SUM_MEAN":0.0,"BURO_STATUS_5_SUM_SUM":0.0,"BURO_STATUS_C_MEAN_MEAN":0.0,"BURO_STATUS_C_MEAN_SUM":0.0,"BURO_STATUS_C_SUM_MEAN":0.0000000001,"BURO_STATUS_C_SUM_SUM":0.0000000006,"BURO_STATUS_X_MEAN_MEAN":0.0003624628,"BURO_STATUS_X_MEAN_SUM":0.0028997024,"BURO_STATUS_X_SUM_MEAN":0.0014498512,"BURO_STATUS_X_SUM_SUM":0.0115988095,"ACTIVE_DAYS_CREDIT_MEAN":-572.5,"ACTIVE_DAYS_CREDIT_MAX":-103.0,"ACTIVE_CREDIT_DAY_OVERDUE_MEAN":0.0,"ACTIVE_CREDIT_DAY_OVERDUE_MAX":0.0,"ACTIVE_DAYS_CREDIT_ENDDATE_MEAN":780.0,"ACTIVE_DAYS_CREDIT_ENDDATE_MAX":780.0,"ACTIVE_DAYS_ENDDATE_FACT_MEAN":0,"ACTIVE_DAYS_ENDDATE_FACT_MAX":0,"ACTIVE_AMT_CREDIT_MAX_OVERDUE_MEAN":40.5,"ACTIVE_AMT_CREDIT_MAX_OVERDUE_MAX":40.5,"ACTIVE_CNT_CREDIT_PROLONG_SUM":0.0,"ACTIVE_AMT_CREDIT_SUM_MEAN":240994.2825,"ACTIVE_AMT_CREDIT_SUM_MAX":450000.0,"ACTIVE_AMT_CREDIT_SUM_DEBT_MEAN":122890.5,"ACTIVE_AMT_CREDIT_SUM_DEBT_MAX":245781.0,"ACTIVE_AMT_CREDIT_SUM_LIMIT_MEAN":15994.2825,"ACTIVE_AMT_CREDIT_SUM_LIMIT_MAX":31988.565,"ACTIVE_AMT_CREDIT_SUM_OVERDUE_MEAN":0.0,"ACTIVE_AMT_CREDIT_SUM_OVERDUE_MAX":0.0,"ACTIVE_DAYS_CREDIT_UPDATE_MEAN":-15.5,"ACTIVE_DAYS_CREDIT_UPDATE_MAX":-7.0,"ACTIVE_AMT_ANNUITY_MEAN":0.0,"ACTIVE_AMT_ANNUITY_MAX":0.0,"ACTIVE_FLAG_MISSING_AMT_ANNUITY_MEAN":0.0,"ACTIVE_FLAG_MISSING_AMT_ANNUITY_MAX":0.0,"ACTIVE_FLAG_MISSING_CREDIT_MAX_OVERDUE_MEAN":0.5,"ACTIVE_FLAG_MISSING_CREDIT_MAX_OVERDUE_MAX":1.0,"ACTIVE_CREDIT_INFO_MISSING_RATIO_MEAN":0.375,"ACTIVE_CREDIT_INFO_MISSING_RATIO_MAX":0.5,"ACTIVE_AMT_CREDIT_DEBT_RATE_MEAN":0.2730893931,"ACTIVE_AMT_CREDIT_DEBT_RATE_MAX":0.5461787863,"ACTIVE_FLAG_ACTIVE_CREDIT_MEAN":1.0,"ACTIVE_FLAG_ACTIVE_CREDIT_MAX":1.0,"ACTIVE_MONTHS_BALANCE_MIN_MIN":-34.0,"ACTIVE_MONTHS_BALANCE_MAX_MAX":0.0,"ACTIVE_MONTHS_BALANCE_SIZE_MEAN":10.0,"ACTIVE_MONTHS_BALANCE_SIZE_MAX":16.0,"CLOSED_DAYS_CREDIT_MEAN":-974.5,"CLOSED_DAYS_CREDIT_MAX":-476.0,"CLOSED_CREDIT_DAY_OVERDUE_MEAN":0.0,"CLOSED_CREDIT_DAY_OVERDUE_MAX":0.0,"CLOSED_DAYS_CREDIT_ENDDATE_MEAN":-574.8,"CLOSED_DAYS_CREDIT_ENDDATE_MAX":85.0,"CLOSED_DAYS_ENDDATE_FACT_MEAN":-697.5,"CLOSED_DAYS_ENDDATE_FACT_MAX":-36.0,"CLOSED_AMT_CREDIT_MAX_OVERDUE_MEAN":2091.16125,"CLOSED_AMT_CREDIT_MAX_OVERDUE_MAX":5043.645,"CLOSED_CNT_CREDIT_PROLONG_SUM":0.0,"CLOSED_AMT_CREDIT_SUM_MEAN":63844.5,"CLOSED_AMT_CREDIT_SUM_MAX":135000.0,"CLOSED_AMT_CREDIT_SUM_DEBT_MEAN":0.0,"CLOSED_AMT_CREDIT_SUM_DEBT_MAX":0.0,"CLOSED_AMT_CREDIT_SUM_LIMIT_MEAN":0.0,"CLOSED_AMT_CREDIT_SUM_LIMIT_MAX":0.0,"CLOSED_AMT_CREDIT_SUM_OVERDUE_MEAN":0.0,"CLOSED_AMT_CREDIT_SUM_OVERDUE_MAX":0.0,"CLOSED_DAYS_CREDIT_UPDATE_MEAN":-661.3333333333,"CLOSED_DAYS_CREDIT_UPDATE_MAX":-34.0,"CLOSED_AMT_ANNUITY_MEAN":0.0,"CLOSED_AMT_ANNUITY_MAX":0.0,"CLOSED_FLAG_MISSING_AMT_ANNUITY_MEAN":0.1666666667,"CLOSED_FLAG_MISSING_AMT_ANNUITY_MAX":1.0,"CLOSED_FLAG_MISSING_CREDIT_MAX_OVERDUE_MEAN":0.3333333333,"CLOSED_FLAG_MISSING_CREDIT_MAX_OVERDUE_MAX":1.0,"CLOSED_CREDIT_INFO_MISSING_RATIO_MEAN":0.3333333333,"CLOSED_CREDIT_INFO_MISSING_RATIO_MAX":0.5,"CLOSED_AMT_CREDIT_DEBT_RATE_MEAN":0.0,"CLOSED_AMT_CREDIT_DEBT_RATE_MAX":0.0,"CLOSED_FLAG_ACTIVE_CREDIT_MEAN":0.0,"CLOSED_FLAG_ACTIVE_CREDIT_MAX":0.0,"CLOSED_MONTHS_BALANCE_MIN_MIN":-47.0,"CLOSED_MONTHS_BALANCE_MAX_MAX":0.0,"CLOSED_MONTHS_BALANCE_SIZE_MEAN":15.0,"CLOSED_MONTHS_BALANCE_SIZE_MAX":22.0,"PREV__TOTAL_MISSING_RATIO":0.0140186916,"APPROVED__TOTAL_MISSING_RATIO":0.0434782609,"INSTAL__TOTAL_MISSING_RATIO":0.0,"CC__TOTAL_MISSING_RATIO":1.0,"POS__TOTAL_MISSING_RATIO":0.0,"BURO__TOTAL_MISSING_RATIO":0.0,"ACTIVE__TOTAL_MISSING_RATIO":0.0512820513,"CLOSED__TOTAL_MISSING_RATIO":0.0}]
}

Overwriting instances.json


#### Build and create container
Create a Dockerfile

In [8]:
%%writefile Dockerfile

FROM tiangolo/uvicorn-gunicorn-fastapi:python3.9

COPY ./app /app
COPY requirements.txt requirements.txt

RUN pip install -r requirements.txt

Overwriting Dockerfile


A repository is created for the docker image.

In [23]:
! gcloud artifacts repositories create {D_REPOSITORY} --repository-format=docker --location={REGION} --description="Docker repository"

! gcloud artifacts repositories list

Create request issued for: [default-clf-repo]
Waiting for operation [projects/rovaja-ml-playground/locations/europe-west1/operations/399532a4-da8f-481d-9952-7de138da57e4] to complete...
...................done.
Created repository [default-clf-repo].


                                                                         ARTIFACT_REGISTRY
REPOSITORY            FORMAT  MODE                 DESCRIPTION        LOCATION      LABELS  ENCRYPTION          CREATE_TIME          UPDATE_TIME          SIZE (MB)
default-clf-repo      DOCKER  STANDARD_REPOSITORY  Docker repository  europe-west1          Google-managed key  2023-09-23T19:10:06  2023-09-23T19:10:06  0
default-xgb-clf-repo  DOCKER  STANDARD_REPOSITORY  Docker repository  europe-west1          Google-managed key  2023-09-23T19:05:28  2023-09-23T19:08:44  969.618


Listing items under project rovaja-ml-playground, across all locations.



Push the image to the created artifact repository using Cloud-Build.

Note: The following command automatically considers the Dockerfile from the directory it is being run from.

In [9]:
!gcloud builds submit --region={REGION} --tag={REGION}-docker.pkg.dev/{PROJECT_ID}/{D_REPOSITORY}/{IMAGE}

----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "e22fc308-fbf3-4bd2-97ae-0f40761796c0"

FETCHSOURCE
Fetching storage object: gs://rovaja-ml-playground_cloudbuild/source/1695485860.08157-00c4a282d0a04aab9969d2e6537fcc27.tgz#1695485867180026
Copying gs://rovaja-ml-playground_cloudbuild/source/1695485860.08157-00c4a282d0a04aab9969d2e6537fcc27.tgz#1695485867180026...
/ [0 files][    0.0 B/  1.1 MiB]                                                
-
- [0 files][  1.0 MiB/  1.1 MiB]                                                
- [1 files][  1.1 MiB/  1.1 MiB]                                                

Operation completed over 1 objects/1.1 MiB.
BUILD
Already have image (with digest): gcr.io/cloud-builders/docker
Sending build context to Docker daemon  2.227MB


Step 1/4 : FROM tiangolo/uvicorn-gunicorn-fastapi:python3.9
python3.9: Pulling from tiangolo/uvicorn-gunicorn-fastapi
012c0b3e998c: Already exists
00046d1e755e: Already exists
9

Creating temporary tarball archive of 13 file(s) totalling 2.1 MiB before compression.
Uploading tarball of [.] to [gs://rovaja-ml-playground_cloudbuild/source/1695485860.08157-00c4a282d0a04aab9969d2e6537fcc27.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/rovaja-ml-playground/locations/europe-west1/builds/e22fc308-fbf3-4bd2-97ae-0f40761796c0].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds;region=europe-west1/e22fc308-fbf3-4bd2-97ae-0f40761796c0?project=8393548517 ].


#### Deploy ML model to Vertex AI

In [10]:
model_to_deploy = aiplatform.Model.upload(
    display_name=MODEL_DISPLAY_NAME,
    artifact_uri=f"{BUCKET_URI}/{MODEL_ARTIFACT_DIR}",
    serving_container_image_uri=f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{D_REPOSITORY}/{IMAGE}",
    location=f"{REGION}",
)

Creating Model
Create Model backing LRO: projects/8393548517/locations/europe-west1/models/2363993980183511040/operations/6289457749285666816
Model created. Resource name: projects/8393548517/locations/europe-west1/models/2363993980183511040@1
To use this Model in another session:
model = aiplatform.Model('projects/8393548517/locations/europe-west1/models/2363993980183511040@1')


Model is deployed into a Vertex AI endoint using the smallest available node.

In [11]:
endpoint = model_to_deploy.deploy(machine_type="n1-standard-2")

Creating Endpoint
Create Endpoint backing LRO: projects/8393548517/locations/europe-west1/endpoints/2198785761040400384/operations/2668563648879788032
Endpoint created. Resource name: projects/8393548517/locations/europe-west1/endpoints/2198785761040400384
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/8393548517/locations/europe-west1/endpoints/2198785761040400384')
Deploying model to Endpoint : projects/8393548517/locations/europe-west1/endpoints/2198785761040400384
Deploy Endpoint model backing LRO: projects/8393548517/locations/europe-west1/endpoints/2198785761040400384/operations/7280249667307175936
Endpoint model deployed. Resource name: projects/8393548517/locations/europe-west1/endpoints/2198785761040400384


In [12]:
endpoint.name

'2198785761040400384'

### Test connection

In [ ]:
endpoint.predict(
    instances=[
        {
            "CODE_GENDER_f": 0,
            "FLAG_OWN_CAR_y": 0,
            "FLAG_OWN_REALTY_y": 1,
            "CNT_CHILDREN": 0,
            "AMT_INCOME_TOTAL": 202500.0,
            "AMT_CREDIT": 406597.5,
            "AMT_ANNUITY": 24700.5,
            "AMT_GOODS_PRICE": 351000.0,
            "REGION_POPULATION_RELATIVE": 0.018801,
            "DAYS_BIRTH": -9461,
            "DAYS_EMPLOYED": -637.0,
            "DAYS_ID_PUBLISH": -2120,
            "OWN_CAR_AGE": 0,
            "CNT_FAM_MEMBERS": 1.0,
            "REGION_RATING_CLIENT": 1,
            "REGION_RATING_CLIENT_W_CITY": 1,
            "REG_REGION_NOT_LIVE_REGION_1": 0,
            "REG_REGION_NOT_WORK_REGION_1": 0,
            "LIVE_REGION_NOT_WORK_REGION_1": 0,
            "REG_CITY_NOT_LIVE_CITY_1": 0,
            "REG_CITY_NOT_WORK_CITY_1": 0,
            "LIVE_CITY_NOT_WORK_CITY_1": 0,
            "EXT_SOURCE_1": 0.0830369674,
            "EXT_SOURCE_2": 0.2629485927,
            "EXT_SOURCE_3": 0.1393757801,
            "APARTMENTS_AVG": 0.0247,
            "BASEMENTAREA_AVG": 0.0369,
            "YEARS_BEGINEXPLUATATION_AVG": 0.9722,
            "YEARS_BUILD_AVG": 0.6192,
            "COMMONAREA_AVG": 0.0143,
            "ELEVATORS_AVG": 0.0,
            "ENTRANCES_AVG": 0.069,
            "FLOORSMAX_AVG": 0.0833,
            "FLOORSMIN_AVG": 0.125,
            "LANDAREA_AVG": 0.0369,
            "LIVINGAPARTMENTS_AVG": 0.0202,
            "LIVINGAREA_AVG": 0.019,
            "NONLIVINGAPARTMENTS_AVG": 0.0,
            "NONLIVINGAREA_AVG": 0.0,
            "APARTMENTS_MODE": 0.0252,
            "BASEMENTAREA_MODE": 0.0383,
            "YEARS_BEGINEXPLUATATION_MODE": 0.9722,
            "YEARS_BUILD_MODE": 0.6341,
            "COMMONAREA_MODE": 0.0144,
            "ELEVATORS_MODE": 0.0,
            "ENTRANCES_MODE": 0.069,
            "FLOORSMAX_MODE": 0.0833,
            "FLOORSMIN_MODE": 0.125,
            "LANDAREA_MODE": 0.0377,
            "LIVINGAPARTMENTS_MODE": 0.022,
            "LIVINGAREA_MODE": 0.0198,
            "NONLIVINGAPARTMENTS_MODE": 0.0,
            "NONLIVINGAREA_MODE": 0.0,
            "APARTMENTS_MEDI": 0.025,
            "BASEMENTAREA_MEDI": 0.0369,
            "YEARS_BEGINEXPLUATATION_MEDI": 0.9722,
            "YEARS_BUILD_MEDI": 0.6243,
            "COMMONAREA_MEDI": 0.0144,
            "ELEVATORS_MEDI": 0.0,
            "ENTRANCES_MEDI": 0.069,
            "FLOORSMAX_MEDI": 0.0833,
            "FLOORSMIN_MEDI": 0.125,
            "LANDAREA_MEDI": 0.0375,
            "LIVINGAPARTMENTS_MEDI": 0.0205,
            "LIVINGAREA_MEDI": 0.0193,
            "NONLIVINGAPARTMENTS_MEDI": 0.0,
            "NONLIVINGAREA_MEDI": 0.0,
            "TOTALAREA_MODE": 0.0149,
            "EMERGENCYSTATE_MODE_yes": 0.0,
            "OBS_30_CNT_SOCIAL_CIRCLE": 2.0,
            "DEF_30_CNT_SOCIAL_CIRCLE": 2.0,
            "OBS_60_CNT_SOCIAL_CIRCLE": 2.0,
            "DEF_60_CNT_SOCIAL_CIRCLE": 2.0,
            "DAYS_LAST_PHONE_CHANGE": -1134.0,
            "FLAG_DOCUMENT_3_1": 1,
            "FLAG_DOCUMENT_5_1": 0,
            "FLAG_DOCUMENT_6_1": 0,
            "FLAG_DOCUMENT_8_1": 0,
            "FLAG_DOCUMENT_9_1": 0,
            "FLAG_DOCUMENT_11_1": 0,
            "FLAG_DOCUMENT_13_1": 0,
            "FLAG_DOCUMENT_14_1": 0,
            "FLAG_DOCUMENT_15_1": 0,
            "FLAG_DOCUMENT_16_1": 0,
            "FLAG_DOCUMENT_18_1": 0,
            "AMT_REQ_CREDIT_BUREAU_HOUR": 0.0,
            "AMT_REQ_CREDIT_BUREAU_DAY": 0.0,
            "AMT_REQ_CREDIT_BUREAU_WEEK": 0.0,
            "AMT_REQ_CREDIT_BUREAU_MON": 0.0,
            "AMT_REQ_CREDIT_BUREAU_QRT": 0.0,
            "AMT_REQ_CREDIT_BUREAU_YEAR": 1.0,
            "HOUSING_INFO_MISSING_RATIO": 0.0,
            "FLAG_CREDIT_BUREAU_MISSING_1": 0,
            "NAME_CONTRACT_TYPE_Cash_loans": 1,
            "NAME_CONTRACT_TYPE_Revolving_loans": 0,
            "NAME_INCOME_TYPE_State_servant": 0,
            "NAME_INCOME_TYPE_Working": 1,
            "NAME_INCOME_TYPE_beneficiary": 0,
            "NAME_INCOME_TYPE_commercial_associate": 0,
            "NAME_EDUCATION_TYPE_Higher_education": 0,
            "NAME_EDUCATION_TYPE_Incomplete_higher": 0,
            "NAME_EDUCATION_TYPE_Lower_secondary": 0,
            "NAME_EDUCATION_TYPE_Secondary___secondary_special": 1,
            "NAME_FAMILY_STATUS_Civil_marriage": 0,
            "NAME_FAMILY_STATUS_Married": 0,
            "NAME_FAMILY_STATUS_Separated": 0,
            "NAME_FAMILY_STATUS_Single___not_married": 1,
            "NAME_FAMILY_STATUS_Widow": 0,
            "NAME_HOUSING_TYPE_Co_op_apartment": 0,
            "NAME_HOUSING_TYPE_House___apartment": 1,
            "NAME_HOUSING_TYPE_Municipal_apartment": 0,
            "NAME_HOUSING_TYPE_Office_apartment": 0,
            "NAME_HOUSING_TYPE_Rented_apartment": 0,
            "NAME_HOUSING_TYPE_With_parents": 0,
            "OCCUPATION_TYPE_Accountants": 0,
            "OCCUPATION_TYPE_Cooking_staff": 0,
            "OCCUPATION_TYPE_Core_staff": 0,
            "OCCUPATION_TYPE_Drivers": 0,
            "OCCUPATION_TYPE_High_skill_tech_staff": 0,
            "OCCUPATION_TYPE_Laborers": 1,
            "OCCUPATION_TYPE_Managers": 0,
            "OCCUPATION_TYPE_Medicine_staff": 0,
            "OCCUPATION_TYPE_Private_service_staff": 0,
            "OCCUPATION_TYPE_Security_staff": 0,
            "OCCUPATION_TYPE_administration_staff": 0,
            "OCCUPATION_TYPE_low_skill_Laborers": 0,
            "OCCUPATION_TYPE_sales_staff": 0,
            "ORGANIZATION_TYPE_Business_Entity": 1,
            "ORGANIZATION_TYPE_Construction": 0,
            "ORGANIZATION_TYPE_Finance": 0,
            "ORGANIZATION_TYPE_House": 0,
            "ORGANIZATION_TYPE_Industry": 0,
            "ORGANIZATION_TYPE_Other": 0,
            "ORGANIZATION_TYPE_Public": 0,
            "ORGANIZATION_TYPE_Self_employed": 0,
            "ORGANIZATION_TYPE_Services": 0,
            "ORGANIZATION_TYPE_Trade": 0,
            "ORGANIZATION_TYPE_Transport": 0,
            "FONDKAPREMONT_MODE_not_specified": 0,
            "FONDKAPREMONT_MODE_org_spec_account": 0,
            "FONDKAPREMONT_MODE_reg_oper_account": 1,
            "FONDKAPREMONT_MODE_reg_oper_spec_account": 0,
            "HOUSETYPE_MODE_block_of_flats": 1,
            "HOUSETYPE_MODE_specific_housing": 0,
            "HOUSETYPE_MODE_terraced_house": 0,
            "WALLSMATERIAL_MODE_Block": 0,
            "WALLSMATERIAL_MODE_Mixed": 0,
            "WALLSMATERIAL_MODE_Monolithic": 0,
            "WALLSMATERIAL_MODE_Others": 0,
            "WALLSMATERIAL_MODE_Panel": 0,
            "WALLSMATERIAL_MODE_Stone__brick": 1,
            "WALLSMATERIAL_MODE_Wooden": 0,
            "PREV_AMT_ANNUITY_MEAN": 0.0000000006,
            "PREV_AMT_ANNUITY_MAX": 0.0000000006,
            "PREV_AMT_APPLICATION_MEAN": 0.0000000124,
            "PREV_AMT_APPLICATION_MAX": 0.0000000124,
            "PREV_AMT_CREDIT_MEAN": 0.0000000124,
            "PREV_AMT_CREDIT_MAX": 0.0000000124,
            "PREV_AMT_DOWN_PAYMENT_MEAN": 0.0,
            "PREV_AMT_DOWN_PAYMENT_MAX": 0.0,
            "PREV_AMT_GOODS_PRICE_MEAN": 0.0000000124,
            "PREV_AMT_GOODS_PRICE_MAX": 0.0000000124,
            "PREV_RATE_DOWN_PAYMENT_MEAN": 0.0,
            "PREV_RATE_DOWN_PAYMENT_MAX": 0.0,
            "PREV_DAYS_DECISION_MEAN": -0.0,
            "PREV_DAYS_DECISION_MAX": -0.0,
            "PREV_SELLERPLACE_AREA_MEAN": 0.0,
            "PREV_SELLERPLACE_AREA_MAX": 0.0,
            "PREV_CNT_PAYMENT_MEAN": 0.0,
            "PREV_CNT_PAYMENT_MAX": 0.0,
            "PREV_DAYS_FIRST_DRAWING_MEAN": 0,
            "PREV_DAYS_FIRST_DRAWING_MAX": 0,
            "PREV_DAYS_FIRST_DUE_MEAN": -0.0,
            "PREV_DAYS_FIRST_DUE_MAX": -0.0,
            "PREV_DAYS_LAST_DUE_1ST_VERSION_MEAN": 0.0,
            "PREV_DAYS_LAST_DUE_1ST_VERSION_MAX": 0.0,
            "PREV_DAYS_LAST_DUE_MEAN": -0.0,
            "PREV_DAYS_LAST_DUE_MAX": -0.0,
            "PREV_DAYS_TERMINATION_MEAN": -0.0,
            "PREV_DAYS_TERMINATION_MAX": -0.0,
            "PREV_FLAG_MISSING_ANNUITY_MEAN": 0.0,
            "PREV_FLAG_MISSING_ANNUITY_MAX": 0.0,
            "PREV_INTEREST_AMT_MEAN": 0.000000003,
            "PREV_INTEREST_AMT_MAX": 0.000000003,
            "PREV_INTEREST_RATIO_MEAN": 0.0,
            "PREV_INTEREST_RATIO_MAX": 0.0,
            "PREV_INTEREST_RATE_MEAN": 0.0,
            "PREV_INTEREST_RATE_MAX": 0.0,
            "PREV_AMT_REFUSED_MEAN": 0.0,
            "PREV_AMT_REFUSED_MAX": 0.0,
            "PREV_AMT_REFUSED_RATIO_MEAN": 0.0,
            "PREV_AMT_REFUSED_RATIO_MAX": 0.0,
            "PREV_AMT_CREDIT_GOODS_RATIO_MEAN": 0.0,
            "PREV_AMT_CREDIT_GOODS_RATIO_MAX": 0.0,
            "PREV_MISSING_VALUES_RATIO_MEAN": 0.0,
            "PREV_MISSING_VALUES_RATIO_MAX": 0.0,
            "PREV_CREDIT_DOWNPAYMENT_RATIO_MEAN": 0.0,
            "PREV_CREDIT_DOWNPAYMENT_RATIO_MAX": 0.0,
            "PREV_NAME_CONTRACT_TYPE_Cash_loans_MEAN": 0.0,
            "PREV_NAME_CONTRACT_TYPE_Cash_loans_SUM": 0.0,
            "PREV_NAME_CONTRACT_TYPE_Consumer_loans_MEAN": 0.0,
            "PREV_NAME_CONTRACT_TYPE_Consumer_loans_SUM": 0.0,
            "PREV_NAME_CONTRACT_TYPE_Revolving_loans_MEAN": 0.0,
            "PREV_NAME_CONTRACT_TYPE_Revolving_loans_SUM": 0.0,
            "PREV_NAME_CASH_LOAN_PURPOSE_Other_MEAN": 0.0,
            "PREV_NAME_CASH_LOAN_PURPOSE_Other_SUM": 0.0,
            "PREV_NAME_CASH_LOAN_PURPOSE_Repairs_MEAN": 0.0,
            "PREV_NAME_CASH_LOAN_PURPOSE_Repairs_SUM": 0.0,
            "PREV_NAME_CASH_LOAN_PURPOSE_Urgent_needs_MEAN": 0.0,
            "PREV_NAME_CASH_LOAN_PURPOSE_Urgent_needs_SUM": 0.0,
            "PREV_NAME_CASH_LOAN_PURPOSE_XAP_MEAN": 0.0,
            "PREV_NAME_CASH_LOAN_PURPOSE_XAP_SUM": 0.0,
            "PREV_NAME_PAYMENT_TYPE_Cash_through_the_bank_MEAN": 0.0,
            "PREV_NAME_PAYMENT_TYPE_Cash_through_the_bank_SUM": 0.0,
            "PREV_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_MEAN": 0.0,
            "PREV_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_SUM": 0.0,
            "PREV_NAME_PAYMENT_TYPE_Non_cash_from_your_account_MEAN": 0.0,
            "PREV_NAME_PAYMENT_TYPE_Non_cash_from_your_account_SUM": 0.0,
            "PREV_CODE_REJECT_REASON_CLIENT_MEAN": 0.0,
            "PREV_CODE_REJECT_REASON_CLIENT_SUM": 0.0,
            "PREV_CODE_REJECT_REASON_HC_MEAN": 0.0,
            "PREV_CODE_REJECT_REASON_HC_SUM": 0.0,
            "PREV_CODE_REJECT_REASON_LIMIT_MEAN": 0.0,
            "PREV_CODE_REJECT_REASON_LIMIT_SUM": 0.0,
            "PREV_CODE_REJECT_REASON_SCO_MEAN": 0.0,
            "PREV_CODE_REJECT_REASON_SCO_SUM": 0.0,
            "PREV_CODE_REJECT_REASON_SCOFR_MEAN": 0.0,
            "PREV_CODE_REJECT_REASON_SCOFR_SUM": 0.0,
            "PREV_CODE_REJECT_REASON_SYSTEM_MEAN": 0.0,
            "PREV_CODE_REJECT_REASON_SYSTEM_SUM": 0.0,
            "PREV_CODE_REJECT_REASON_VERIF_MEAN": 0.0,
            "PREV_CODE_REJECT_REASON_VERIF_SUM": 0.0,
            "PREV_CODE_REJECT_REASON_XAP_MEAN": 0.0,
            "PREV_CODE_REJECT_REASON_XAP_SUM": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Audio_Video_MEAN": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Audio_Video_SUM": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Auto_Accessories_MEAN": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Auto_Accessories_SUM": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Clothing_and_Accessories_MEAN": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Clothing_and_Accessories_SUM": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Computers_MEAN": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Computers_SUM": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Construction_Materials_MEAN": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Construction_Materials_SUM": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Consumer_Electronics_MEAN": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Consumer_Electronics_SUM": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Furniture_MEAN": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Furniture_SUM": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Gardening_MEAN": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Gardening_SUM": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Homewares_MEAN": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Homewares_SUM": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Jewelry_MEAN": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Jewelry_SUM": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Medical_Supplies_MEAN": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Medical_Supplies_SUM": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Medicine_MEAN": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Medicine_SUM": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Mobile_MEAN": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Mobile_SUM": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Office_Appliances_MEAN": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Office_Appliances_SUM": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Other_MEAN": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Other_SUM": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Photo___Cinema_Equipment_MEAN": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Photo___Cinema_Equipment_SUM": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Sport_and_Leisure_MEAN": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Sport_and_Leisure_SUM": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Tourism_MEAN": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Tourism_SUM": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Vehicles_MEAN": 0.0,
            "PREV_NAME_GOODS_CATEGORY_Vehicles_SUM": 0.0,
            "PREV_NAME_PORTFOLIO_Cards_MEAN": 0.0,
            "PREV_NAME_PORTFOLIO_Cards_SUM": 0.0,
            "PREV_NAME_PORTFOLIO_Cars_MEAN": 0.0,
            "PREV_NAME_PORTFOLIO_Cars_SUM": 0.0,
            "PREV_NAME_PORTFOLIO_Cash_MEAN": 0.0,
            "PREV_NAME_PORTFOLIO_Cash_SUM": 0.0,
            "PREV_NAME_PORTFOLIO_POS_MEAN": 0.0,
            "PREV_NAME_PORTFOLIO_POS_SUM": 0.0,
            "PREV_CHANNEL_TYPE_AP___Cash_loan__MEAN": 0.0,
            "PREV_CHANNEL_TYPE_AP___Cash_loan__SUM": 0.0,
            "PREV_CHANNEL_TYPE_Car_dealer_MEAN": 0.0,
            "PREV_CHANNEL_TYPE_Car_dealer_SUM": 0.0,
            "PREV_CHANNEL_TYPE_Contact_center_MEAN": 0.0,
            "PREV_CHANNEL_TYPE_Contact_center_SUM": 0.0,
            "PREV_CHANNEL_TYPE_Country_wide_MEAN": 0.0,
            "PREV_CHANNEL_TYPE_Country_wide_SUM": 0.0,
            "PREV_CHANNEL_TYPE_Credit_and_cash_offices_MEAN": 0.0,
            "PREV_CHANNEL_TYPE_Credit_and_cash_offices_SUM": 0.0,
            "PREV_CHANNEL_TYPE_Regional___Local_MEAN": 0.0,
            "PREV_CHANNEL_TYPE_Regional___Local_SUM": 0.0,
            "PREV_CHANNEL_TYPE_Stone_MEAN": 0.0,
            "PREV_CHANNEL_TYPE_Stone_SUM": 0.0,
            "PREV_NAME_SELLER_INDUSTRY_Auto_technology_MEAN": 0.0,
            "PREV_NAME_SELLER_INDUSTRY_Auto_technology_SUM": 0.0,
            "PREV_NAME_SELLER_INDUSTRY_Clothing_MEAN": 0.0,
            "PREV_NAME_SELLER_INDUSTRY_Clothing_SUM": 0.0,
            "PREV_NAME_SELLER_INDUSTRY_Connectivity_MEAN": 0.0,
            "PREV_NAME_SELLER_INDUSTRY_Connectivity_SUM": 0.0,
            "PREV_NAME_SELLER_INDUSTRY_Construction_MEAN": 0.0,
            "PREV_NAME_SELLER_INDUSTRY_Construction_SUM": 0.0,
            "PREV_NAME_SELLER_INDUSTRY_Consumer_electronics_MEAN": 0.0,
            "PREV_NAME_SELLER_INDUSTRY_Consumer_electronics_SUM": 0.0,
            "PREV_NAME_SELLER_INDUSTRY_Furniture_MEAN": 0.0,
            "PREV_NAME_SELLER_INDUSTRY_Furniture_SUM": 0.0,
            "PREV_NAME_SELLER_INDUSTRY_Industry_MEAN": 0.0,
            "PREV_NAME_SELLER_INDUSTRY_Industry_SUM": 0.0,
            "PREV_NAME_SELLER_INDUSTRY_Jewelry_MEAN": 0.0,
            "PREV_NAME_SELLER_INDUSTRY_Jewelry_SUM": 0.0,
            "PREV_NAME_SELLER_INDUSTRY_MLM_partners_MEAN": 0.0,
            "PREV_NAME_SELLER_INDUSTRY_MLM_partners_SUM": 0.0,
            "PREV_NAME_SELLER_INDUSTRY_Tourism_MEAN": 0.0,
            "PREV_NAME_SELLER_INDUSTRY_Tourism_SUM": 0.0,
            "PREV_PRODUCT_COMBINATION_Card_Street_MEAN": 0.0,
            "PREV_PRODUCT_COMBINATION_Card_Street_SUM": 0.0,
            "PREV_PRODUCT_COMBINATION_Card_X_Sell_MEAN": 0.0,
            "PREV_PRODUCT_COMBINATION_Card_X_Sell_SUM": 0.0,
            "PREV_PRODUCT_COMBINATION_Cash_MEAN": 0.0,
            "PREV_PRODUCT_COMBINATION_Cash_SUM": 0.0,
            "PREV_PRODUCT_COMBINATION_Cash_Street__high_MEAN": 0.0,
            "PREV_PRODUCT_COMBINATION_Cash_Street__high_SUM": 0.0,
            "PREV_PRODUCT_COMBINATION_Cash_Street__low_MEAN": 0.0,
            "PREV_PRODUCT_COMBINATION_Cash_Street__low_SUM": 0.0,
            "PREV_PRODUCT_COMBINATION_Cash_Street__middle_MEAN": 0.0,
            "PREV_PRODUCT_COMBINATION_Cash_Street__middle_SUM": 0.0,
            "PREV_PRODUCT_COMBINATION_Cash_X_Sell__high_MEAN": 0.0,
            "PREV_PRODUCT_COMBINATION_Cash_X_Sell__high_SUM": 0.0,
            "PREV_PRODUCT_COMBINATION_Cash_X_Sell__low_MEAN": 0.0,
            "PREV_PRODUCT_COMBINATION_Cash_X_Sell__low_SUM": 0.0,
            "PREV_PRODUCT_COMBINATION_Cash_X_Sell__middle_MEAN": 0.0,
            "PREV_PRODUCT_COMBINATION_Cash_X_Sell__middle_SUM": 0.0,
            "PREV_PRODUCT_COMBINATION_POS_household_with_interest_MEAN": 0.0,
            "PREV_PRODUCT_COMBINATION_POS_household_with_interest_SUM": 0.0,
            "PREV_PRODUCT_COMBINATION_POS_household_without_interest_MEAN": 0.0,
            "PREV_PRODUCT_COMBINATION_POS_household_without_interest_SUM": 0.0,
            "PREV_PRODUCT_COMBINATION_POS_industry_with_interest_MEAN": 0.0,
            "PREV_PRODUCT_COMBINATION_POS_industry_with_interest_SUM": 0.0,
            "PREV_PRODUCT_COMBINATION_POS_industry_without_interest_MEAN": 0.0,
            "PREV_PRODUCT_COMBINATION_POS_industry_without_interest_SUM": 0.0,
            "PREV_PRODUCT_COMBINATION_POS_mobile_with_interest_MEAN": 0.0,
            "PREV_PRODUCT_COMBINATION_POS_mobile_with_interest_SUM": 0.0,
            "PREV_PRODUCT_COMBINATION_POS_mobile_without_interest_MEAN": 0.0,
            "PREV_PRODUCT_COMBINATION_POS_mobile_without_interest_SUM": 0.0,
            "PREV_PRODUCT_COMBINATION_POS_other_with_interest_MEAN": 0.0,
            "PREV_PRODUCT_COMBINATION_POS_other_with_interest_SUM": 0.0,
            "PREV_PRODUCT_COMBINATION_POS_others_without_interest_MEAN": 0.0,
            "PREV_PRODUCT_COMBINATION_POS_others_without_interest_SUM": 0.0,
            "PREV_NAME_CONTRACT_STATUS_MEAN": 0.0,
            "PREV_NAME_CONTRACT_STATUS_SUM": 0.0,
            "PREV_NAME_CLIENT_TYPE_MEAN": 0.0,
            "PREV_NAME_CLIENT_TYPE_SUM": 0.0,
            "PREV_NAME_PRODUCT_TYPE_x_sell_MEAN": 0,
            "PREV_NAME_PRODUCT_TYPE_x_sell_SUM": 0.0,
            "PREV_NAME_YIELD_GROUP_MEAN": 0.0,
            "PREV_NAME_YIELD_GROUP_SUM": 0.0,
            "PREV_FLAG_LAST_APPL_PER_CONTRACT_y_MEAN": 0.0,
            "PREV_FLAG_LAST_APPL_PER_CONTRACT_y_SUM": 0.0,
            "PREV_NFLAG_LAST_APPL_IN_DAY_1_MEAN": 0.0,
            "PREV_NFLAG_LAST_APPL_IN_DAY_1_SUM": 0.0,
            "PREV_NFLAG_INSURED_ON_APPROVAL_1_0_MEAN": 0.0,
            "PREV_NFLAG_INSURED_ON_APPROVAL_1_0_SUM": 0.0,
            "PREV_FLAG_MISSING_FIRST_DRAW_1_MEAN": 0.0,
            "PREV_FLAG_MISSING_FIRST_DRAW_1_SUM": 0.0,
            "PREV_REFUSED_APPLICATION_1_MEAN": 0.0,
            "PREV_REFUSED_APPLICATION_1_SUM": 0.0,
            "APPROVED_AMT_ANNUITY_MEAN": 9251.775,
            "APPROVED_AMT_ANNUITY_MAX": 9251.775,
            "APPROVED_AMT_APPLICATION_MEAN": 179055.0,
            "APPROVED_AMT_APPLICATION_MAX": 179055.0,
            "APPROVED_AMT_CREDIT_MEAN": 179055.0,
            "APPROVED_AMT_CREDIT_MAX": 179055.0,
            "APPROVED_AMT_DOWN_PAYMENT_MEAN": 0.0,
            "APPROVED_AMT_DOWN_PAYMENT_MAX": 0.0,
            "APPROVED_AMT_GOODS_PRICE_MEAN": 179055.0,
            "APPROVED_AMT_GOODS_PRICE_MAX": 179055.0,
            "APPROVED_RATE_DOWN_PAYMENT_MEAN": 0.0,
            "APPROVED_RATE_DOWN_PAYMENT_MAX": 0.0,
            "APPROVED_DAYS_DECISION_MEAN": -606.0,
            "APPROVED_DAYS_DECISION_MAX": -606.0,
            "APPROVED_SELLERPLACE_AREA_MEAN": 500.0,
            "APPROVED_SELLERPLACE_AREA_MAX": 500.0,
            "APPROVED_CNT_PAYMENT_MEAN": 24.0,
            "APPROVED_CNT_PAYMENT_MAX": 24.0,
            "APPROVED_DAYS_FIRST_DRAWING_MEAN": 0,
            "APPROVED_DAYS_FIRST_DRAWING_MAX": 0,
            "APPROVED_DAYS_FIRST_DUE_MEAN": -565.0,
            "APPROVED_DAYS_FIRST_DUE_MAX": -565.0,
            "APPROVED_DAYS_LAST_DUE_1ST_VERSION_MEAN": 125.0,
            "APPROVED_DAYS_LAST_DUE_1ST_VERSION_MAX": 125.0,
            "APPROVED_DAYS_LAST_DUE_MEAN": -25.0,
            "APPROVED_DAYS_LAST_DUE_MAX": -25.0,
            "APPROVED_DAYS_TERMINATION_MEAN": -17.0,
            "APPROVED_DAYS_TERMINATION_MAX": -17.0,
            "APPROVED_FLAG_MISSING_ANNUITY_MEAN": 0.0,
            "APPROVED_FLAG_MISSING_ANNUITY_MAX": 0.0,
            "APPROVED_INTEREST_AMT_MEAN": 42987.6,
            "APPROVED_INTEREST_AMT_MAX": 42987.6,
            "APPROVED_INTEREST_RATIO_MEAN": 0.2400804222,
            "APPROVED_INTEREST_RATIO_MAX": 0.2400804222,
            "APPROVED_INTEREST_RATE_MEAN": 0.2304772053,
            "APPROVED_INTEREST_RATE_MAX": 0.2304772053,
            "APPROVED_AMT_REFUSED_MEAN": 0.0,
            "APPROVED_AMT_REFUSED_MAX": 0.0,
            "APPROVED_AMT_REFUSED_RATIO_MEAN": 1.0,
            "APPROVED_AMT_REFUSED_RATIO_MAX": 1.0,
            "APPROVED_AMT_CREDIT_GOODS_RATIO_MEAN": 1.0,
            "APPROVED_AMT_CREDIT_GOODS_RATIO_MAX": 1.0,
            "APPROVED_MISSING_VALUES_RATIO_MEAN": 0.075,
            "APPROVED_MISSING_VALUES_RATIO_MAX": 0.075,
            "APPROVED_CREDIT_DOWNPAYMENT_RATIO_MEAN": 0.0,
            "APPROVED_CREDIT_DOWNPAYMENT_RATIO_MAX": 0.0,
            "INSTAL_NUM_INSTALMENT_VERSION_NUNIQUE": 19.0,
            "INSTAL_DAYS_INSTALMENT_MAX": -0.0000000003,
            "INSTAL_DAYS_INSTALMENT_MEAN": -0.6525031716,
            "INSTAL_DAYS_ENTRY_PAYMENT_MAX": -0.0000000003,
            "INSTAL_DAYS_ENTRY_PAYMENT_MEAN": -1.0690681524,
            "INSTAL_AMT_INSTALMENT_MAX": 15211.6126257717,
            "INSTAL_AMT_INSTALMENT_MEAN": 840.680640236,
            "INSTAL_AMT_PAYMENT_MAX": 15211.6126257717,
            "INSTAL_AMT_PAYMENT_MEAN": 840.680640236,
            "INSTAL_DAYS_PASS_DUE_MAX": 0.0,
            "INSTAL_DAYS_PASS_DUE_MEAN": 0.0,
            "INSTAL_DAYS_PASS_DUE_SUM": 0.0,
            "INSTAL_DAYS_BEFORE_DUE_MAX": 6.8761151246,
            "INSTAL_DAYS_BEFORE_DUE_MEAN": 0.4165649809,
            "INSTAL_DAYS_BEFORE_DUE_SUM": 7.9147346363,
            "INSTAL_PAYMENT_INST_RATIO_MAX": 0.2865047969,
            "INSTAL_PAYMENT_INST_RATIO_MEAN": 0.0194102011,
            "INSTAL_INST_PAYMENT_DIFFERENCE_MAX": 0.0,
            "INSTAL_INST_PAYMENT_DIFFERENCE_MEAN": 0.0,
            "INSTAL_COUNT": 19.0,
            "CC_MONTHS_BALANCE_MAX": 0,
            "CC_MONTHS_BALANCE_MEAN": 0,
            "CC_AMT_BALANCE_MAX": 0,
            "CC_AMT_BALANCE_MEAN": 0,
            "CC_AMT_CREDIT_LIMIT_ACTUAL_MAX": 0,
            "CC_AMT_CREDIT_LIMIT_ACTUAL_MEAN": 0,
            "CC_AMT_DRAWINGS_ATM_CURRENT_MAX": 0,
            "CC_AMT_DRAWINGS_ATM_CURRENT_MEAN": 0,
            "CC_AMT_DRAWINGS_CURRENT_MAX": 0,
            "CC_AMT_DRAWINGS_CURRENT_MEAN": 0,
            "CC_AMT_DRAWINGS_OTHER_CURRENT_MAX": 0,
            "CC_AMT_DRAWINGS_OTHER_CURRENT_MEAN": 0,
            "CC_AMT_DRAWINGS_POS_CURRENT_MAX": 0,
            "CC_AMT_DRAWINGS_POS_CURRENT_MEAN": 0,
            "CC_AMT_INST_MIN_REGULARITY_MAX": 0,
            "CC_AMT_INST_MIN_REGULARITY_MEAN": 0,
            "CC_AMT_PAYMENT_CURRENT_MAX": 0,
            "CC_AMT_PAYMENT_CURRENT_MEAN": 0,
            "CC_AMT_PAYMENT_TOTAL_CURRENT_MAX": 0,
            "CC_AMT_PAYMENT_TOTAL_CURRENT_MEAN": 0,
            "CC_AMT_RECEIVABLE_PRINCIPAL_MAX": 0,
            "CC_AMT_RECEIVABLE_PRINCIPAL_MEAN": 0,
            "CC_AMT_RECIVABLE_MAX": 0,
            "CC_AMT_RECIVABLE_MEAN": 0,
            "CC_AMT_TOTAL_RECEIVABLE_MAX": 0,
            "CC_AMT_TOTAL_RECEIVABLE_MEAN": 0,
            "CC_CNT_DRAWINGS_ATM_CURRENT_MEAN": 0,
            "CC_CNT_DRAWINGS_ATM_CURRENT_SUM": 0,
            "CC_CNT_DRAWINGS_CURRENT_MEAN": 0,
            "CC_CNT_DRAWINGS_CURRENT_SUM": 0,
            "CC_CNT_DRAWINGS_OTHER_CURRENT_MEAN": 0,
            "CC_CNT_DRAWINGS_OTHER_CURRENT_SUM": 0,
            "CC_CNT_DRAWINGS_POS_CURRENT_MEAN": 0,
            "CC_CNT_DRAWINGS_POS_CURRENT_SUM": 0,
            "CC_CNT_INSTALMENT_MATURE_CUM_MEAN": 0,
            "CC_CNT_INSTALMENT_MATURE_CUM_SUM": 0,
            "CC_SK_DPD_MAX": 0,
            "CC_SK_DPD_MEAN": 0,
            "CC_SK_DPD_DEF_MAX": 0,
            "CC_SK_DPD_DEF_MEAN": 0,
            "CC_NAME_CONTRACT_STATUS_Active_MEAN": 0,
            "CC_NAME_CONTRACT_STATUS_Active_SUM": 0,
            "CC_NAME_CONTRACT_STATUS_Approved_MEAN": 0,
            "CC_NAME_CONTRACT_STATUS_Approved_SUM": 0,
            "CC_NAME_CONTRACT_STATUS_Completed_MEAN": 0,
            "CC_NAME_CONTRACT_STATUS_Completed_SUM": 0,
            "CC_NAME_CONTRACT_STATUS_Demand_MEAN": 0,
            "CC_NAME_CONTRACT_STATUS_Demand_SUM": 0,
            "CC_NAME_CONTRACT_STATUS_Refused_MEAN": 0,
            "CC_NAME_CONTRACT_STATUS_Refused_SUM": 0,
            "CC_NAME_CONTRACT_STATUS_Sent_proposal_MEAN": 0,
            "CC_NAME_CONTRACT_STATUS_Sent_proposal_SUM": 0,
            "CC_NAME_CONTRACT_STATUS_Signed_MEAN": 0,
            "CC_NAME_CONTRACT_STATUS_Signed_SUM": 0,
            "CC_FLAG_MISSING_INST_1_MEAN": 0,
            "CC_FLAG_MISSING_INST_1_SUM": 0,
            "CC_FLAG_MISSING_DRAWING_1_MEAN": 0,
            "CC_FLAG_MISSING_DRAWING_1_SUM": 0,
            "CC_COUNT": 0,
            "POS_MONTHS_BALANCE_MAX": -0.9512294245,
            "POS_MONTHS_BALANCE_MEAN": -5.3649645884,
            "POS_CNT_INSTALMENT_MEAN": 15.1087644403,
            "POS_CNT_INSTALMENT_SUM": 287.0665243659,
            "POS_CNT_INSTALMENT_FUTURE_MEAN": 8.5126238468,
            "POS_CNT_INSTALMENT_FUTURE_SUM": 161.7398530886,
            "POS_SK_DPD_MAX": 0.0,
            "POS_SK_DPD_MEAN": 0.0,
            "POS_SK_DPD_DEF_MAX": 0.0,
            "POS_SK_DPD_DEF_MEAN": 0.0,
            "POS_NAME_CONTRACT_STATUS_Active_MEAN": 0.6295318517,
            "POS_NAME_CONTRACT_STATUS_Active_SUM": 11.9611051819,
            "POS_NAME_CONTRACT_STATUS_Amortized_debt_MEAN": 0.0,
            "POS_NAME_CONTRACT_STATUS_Amortized_debt_SUM": 0.0,
            "POS_NAME_CONTRACT_STATUS_Approved_MEAN": 0.0,
            "POS_NAME_CONTRACT_STATUS_Approved_SUM": 0.0,
            "POS_NAME_CONTRACT_STATUS_Canceled_MEAN": 0.0,
            "POS_NAME_CONTRACT_STATUS_Canceled_SUM": 0.0,
            "POS_NAME_CONTRACT_STATUS_Completed_MEAN": 0.0,
            "POS_NAME_CONTRACT_STATUS_Completed_SUM": 0.0,
            "POS_NAME_CONTRACT_STATUS_Demand_MEAN": 0.0,
            "POS_NAME_CONTRACT_STATUS_Demand_SUM": 0.0,
            "POS_NAME_CONTRACT_STATUS_Returned_to_the_store_MEAN": 0.0,
            "POS_NAME_CONTRACT_STATUS_Returned_to_the_store_SUM": 0.0,
            "POS_NAME_CONTRACT_STATUS_Signed_MEAN": 0.0,
            "POS_NAME_CONTRACT_STATUS_Signed_SUM": 0.0,
            "POS_NAME_CONTRACT_STATUS_XNA_MEAN": 0.0,
            "POS_NAME_CONTRACT_STATUS_XNA_SUM": 0.0,
            "POS_COUNT": 19.0,
            "BURO_DAYS_CREDIT_MEAN": -0.0746673386,
            "BURO_DAYS_CREDIT_MAX": -8.982521045e-29,
            "BURO_CREDIT_DAY_OVERDUE_MEAN": 0.0,
            "BURO_CREDIT_DAY_OVERDUE_MAX": 0.0,
            "BURO_DAYS_CREDIT_ENDDATE_MEAN": 0.0,
            "BURO_DAYS_CREDIT_ENDDATE_MAX": 0.0,
            "BURO_DAYS_ENDDATE_FACT_MEAN": -0.0000000004,
            "BURO_DAYS_ENDDATE_FACT_MAX": -7.407298148e-29,
            "BURO_AMT_CREDIT_MAX_OVERDUE_MEAN": 0.0469751783,
            "BURO_AMT_CREDIT_MAX_OVERDUE_MAX": 0.2348758914,
            "BURO_CNT_CREDIT_PROLONG_SUM": 0.0,
            "BURO_AMT_CREDIT_SUM_MEAN": 23.1893293834,
            "BURO_AMT_CREDIT_SUM_MAX": 185.5146350659,
            "BURO_AMT_CREDIT_SUM_DEBT_MEAN": 1.161009559e-18,
            "BURO_AMT_CREDIT_SUM_DEBT_MAX": 5.805047794e-18,
            "BURO_AMT_CREDIT_SUM_LIMIT_MEAN": 46.3786587665,
            "BURO_AMT_CREDIT_SUM_LIMIT_MAX": 185.5146350659,
            "BURO_AMT_CREDIT_SUM_OVERDUE_MEAN": 0.0,
            "BURO_AMT_CREDIT_SUM_OVERDUE_MAX": 0.0,
            "BURO_DAYS_CREDIT_UPDATE_MEAN": -0.0173982145,
            "BURO_DAYS_CREDIT_UPDATE_MAX": -7.407298148e-29,
            "BURO_AMT_ANNUITY_MEAN": 0.0,
            "BURO_AMT_ANNUITY_MAX": 0.0,
            "BURO_FLAG_MISSING_AMT_ANNUITY_MEAN": 0.0,
            "BURO_FLAG_MISSING_AMT_ANNUITY_MAX": 0.0,
            "BURO_FLAG_MISSING_CREDIT_MAX_OVERDUE_MEAN": 0.0,
            "BURO_FLAG_MISSING_CREDIT_MAX_OVERDUE_MAX": 0.0,
            "BURO_CREDIT_INFO_MISSING_RATIO_MEAN": 0.0003624628,
            "BURO_CREDIT_INFO_MISSING_RATIO_MAX": 0.0028997024,
            "BURO_AMT_CREDIT_DEBT_RATE_MEAN": 2.580015509e-24,
            "BURO_AMT_CREDIT_DEBT_RATE_MAX": 1.290007754e-23,
            "BURO_FLAG_ACTIVE_CREDIT_MEAN": 0.0007249256,
            "BURO_FLAG_ACTIVE_CREDIT_MAX": 0.0057994047,
            "BURO_MONTHS_BALANCE_MIN_MIN": -0.0173982142,
            "BURO_MONTHS_BALANCE_MAX_MAX": 0.0,
            "BURO_MONTHS_BALANCE_SIZE_MEAN": 0.0028997025,
            "BURO_MONTHS_BALANCE_SIZE_MAX": 0.0231976189,
            "BURO_CREDIT_TYPE_Car_loan_MEAN": 0.0,
            "BURO_CREDIT_TYPE_Car_loan_SUM": 0.0,
            "BURO_CREDIT_TYPE_Consumer_credit_MEAN": 0.0,
            "BURO_CREDIT_TYPE_Consumer_credit_SUM": 0.0,
            "BURO_CREDIT_TYPE_Credit_card_MEAN": 0.0007249256,
            "BURO_CREDIT_TYPE_Credit_card_SUM": 0.0057994048,
            "BURO_CREDIT_TYPE_Loan_for_business_development_MEAN": 0.0,
            "BURO_CREDIT_TYPE_Loan_for_business_development_SUM": 0.0,
            "BURO_CREDIT_TYPE_Microloan_MEAN": 0.0,
            "BURO_CREDIT_TYPE_Microloan_SUM": 0.0,
            "BURO_CREDIT_TYPE_Mortgage_MEAN": 0.0,
            "BURO_CREDIT_TYPE_Mortgage_SUM": 0.0,
            "BURO_CREDIT_TYPE_Other_MEAN": 0.0,
            "BURO_CREDIT_TYPE_Other_SUM": 0.0,
            "BURO_CREDIT_CURRENCY_currency_1_MEAN": 0.0007249256,
            "BURO_CREDIT_CURRENCY_currency_1_SUM": 0.0057994048,
            "BURO_CREDIT_CURRENCY_currency_2_MEAN": 0.0,
            "BURO_CREDIT_CURRENCY_currency_2_SUM": 0.0,
            "BURO_CREDIT_CURRENCY_currency_3_MEAN": 0.0,
            "BURO_CREDIT_CURRENCY_currency_3_SUM": 0.0,
            "BURO_CREDIT_CURRENCY_currency_4_MEAN": 0.0,
            "BURO_CREDIT_CURRENCY_currency_4_SUM": 0.0,
            "BURO_CREDIT_ACTIVE_Active_MEAN": 0.0007249256,
            "BURO_CREDIT_ACTIVE_Active_SUM": 0.0057994047,
            "BURO_CREDIT_ACTIVE_Bad_debt_MEAN": 0.0,
            "BURO_CREDIT_ACTIVE_Bad_debt_SUM": 0.0,
            "BURO_CREDIT_ACTIVE_Closed_MEAN": 0.0,
            "BURO_CREDIT_ACTIVE_Closed_SUM": 0.0,
            "BURO_CREDIT_ACTIVE_Sold_MEAN": 0.0,
            "BURO_CREDIT_ACTIVE_Sold_SUM": 0.0,
            "BURO_STATUS_0_MEAN_MEAN": 0.0003624628,
            "BURO_STATUS_0_MEAN_SUM": 0.0028997024,
            "BURO_STATUS_0_SUM_MEAN": 0.0014498512,
            "BURO_STATUS_0_SUM_SUM": 0.0115988096,
            "BURO_STATUS_1_MEAN_MEAN": 6.164288801e-16,
            "BURO_STATUS_1_MEAN_SUM": 0.0,
            "BURO_STATUS_1_SUM_MEAN": 0.0,
            "BURO_STATUS_1_SUM_SUM": 0.0,
            "BURO_STATUS_2_MEAN_MEAN": 0.0,
            "BURO_STATUS_2_MEAN_SUM": 0.0,
            "BURO_STATUS_2_SUM_MEAN": 0.0,
            "BURO_STATUS_2_SUM_SUM": 0.0,
            "BURO_STATUS_3_MEAN_MEAN": 0.0,
            "BURO_STATUS_3_MEAN_SUM": 0.0,
            "BURO_STATUS_3_SUM_MEAN": 0.0,
            "BURO_STATUS_3_SUM_SUM": 0.0,
            "BURO_STATUS_4_MEAN_MEAN": 0.0,
            "BURO_STATUS_4_MEAN_SUM": 0.0,
            "BURO_STATUS_4_SUM_MEAN": 0.0,
            "BURO_STATUS_4_SUM_SUM": 0.0,
            "BURO_STATUS_5_MEAN_MEAN": 0.0,
            "BURO_STATUS_5_MEAN_SUM": 0.0,
            "BURO_STATUS_5_SUM_MEAN": 0.0,
            "BURO_STATUS_5_SUM_SUM": 0.0,
            "BURO_STATUS_C_MEAN_MEAN": 0.0,
            "BURO_STATUS_C_MEAN_SUM": 0.0,
            "BURO_STATUS_C_SUM_MEAN": 0.0000000001,
            "BURO_STATUS_C_SUM_SUM": 0.0000000006,
            "BURO_STATUS_X_MEAN_MEAN": 0.0003624628,
            "BURO_STATUS_X_MEAN_SUM": 0.0028997024,
            "BURO_STATUS_X_SUM_MEAN": 0.0014498512,
            "BURO_STATUS_X_SUM_SUM": 0.0115988095,
            "ACTIVE_DAYS_CREDIT_MEAN": -572.5,
            "ACTIVE_DAYS_CREDIT_MAX": -103.0,
            "ACTIVE_CREDIT_DAY_OVERDUE_MEAN": 0.0,
            "ACTIVE_CREDIT_DAY_OVERDUE_MAX": 0.0,
            "ACTIVE_DAYS_CREDIT_ENDDATE_MEAN": 780.0,
            "ACTIVE_DAYS_CREDIT_ENDDATE_MAX": 780.0,
            "ACTIVE_DAYS_ENDDATE_FACT_MEAN": 0,
            "ACTIVE_DAYS_ENDDATE_FACT_MAX": 0,
            "ACTIVE_AMT_CREDIT_MAX_OVERDUE_MEAN": 40.5,
            "ACTIVE_AMT_CREDIT_MAX_OVERDUE_MAX": 40.5,
            "ACTIVE_CNT_CREDIT_PROLONG_SUM": 0.0,
            "ACTIVE_AMT_CREDIT_SUM_MEAN": 240994.2825,
            "ACTIVE_AMT_CREDIT_SUM_MAX": 450000.0,
            "ACTIVE_AMT_CREDIT_SUM_DEBT_MEAN": 122890.5,
            "ACTIVE_AMT_CREDIT_SUM_DEBT_MAX": 245781.0,
            "ACTIVE_AMT_CREDIT_SUM_LIMIT_MEAN": 15994.2825,
            "ACTIVE_AMT_CREDIT_SUM_LIMIT_MAX": 31988.565,
            "ACTIVE_AMT_CREDIT_SUM_OVERDUE_MEAN": 0.0,
            "ACTIVE_AMT_CREDIT_SUM_OVERDUE_MAX": 0.0,
            "ACTIVE_DAYS_CREDIT_UPDATE_MEAN": -15.5,
            "ACTIVE_DAYS_CREDIT_UPDATE_MAX": -7.0,
            "ACTIVE_AMT_ANNUITY_MEAN": 0.0,
            "ACTIVE_AMT_ANNUITY_MAX": 0.0,
            "ACTIVE_FLAG_MISSING_AMT_ANNUITY_MEAN": 0.0,
            "ACTIVE_FLAG_MISSING_AMT_ANNUITY_MAX": 0.0,
            "ACTIVE_FLAG_MISSING_CREDIT_MAX_OVERDUE_MEAN": 0.5,
            "ACTIVE_FLAG_MISSING_CREDIT_MAX_OVERDUE_MAX": 1.0,
            "ACTIVE_CREDIT_INFO_MISSING_RATIO_MEAN": 0.375,
            "ACTIVE_CREDIT_INFO_MISSING_RATIO_MAX": 0.5,
            "ACTIVE_AMT_CREDIT_DEBT_RATE_MEAN": 0.2730893931,
            "ACTIVE_AMT_CREDIT_DEBT_RATE_MAX": 0.5461787863,
            "ACTIVE_FLAG_ACTIVE_CREDIT_MEAN": 1.0,
            "ACTIVE_FLAG_ACTIVE_CREDIT_MAX": 1.0,
            "ACTIVE_MONTHS_BALANCE_MIN_MIN": -34.0,
            "ACTIVE_MONTHS_BALANCE_MAX_MAX": 0.0,
            "ACTIVE_MONTHS_BALANCE_SIZE_MEAN": 10.0,
            "ACTIVE_MONTHS_BALANCE_SIZE_MAX": 16.0,
            "CLOSED_DAYS_CREDIT_MEAN": -974.5,
            "CLOSED_DAYS_CREDIT_MAX": -476.0,
            "CLOSED_CREDIT_DAY_OVERDUE_MEAN": 0.0,
            "CLOSED_CREDIT_DAY_OVERDUE_MAX": 0.0,
            "CLOSED_DAYS_CREDIT_ENDDATE_MEAN": -574.8,
            "CLOSED_DAYS_CREDIT_ENDDATE_MAX": 85.0,
            "CLOSED_DAYS_ENDDATE_FACT_MEAN": -697.5,
            "CLOSED_DAYS_ENDDATE_FACT_MAX": -36.0,
            "CLOSED_AMT_CREDIT_MAX_OVERDUE_MEAN": 2091.16125,
            "CLOSED_AMT_CREDIT_MAX_OVERDUE_MAX": 5043.645,
            "CLOSED_CNT_CREDIT_PROLONG_SUM": 0.0,
            "CLOSED_AMT_CREDIT_SUM_MEAN": 63844.5,
            "CLOSED_AMT_CREDIT_SUM_MAX": 135000.0,
            "CLOSED_AMT_CREDIT_SUM_DEBT_MEAN": 0.0,
            "CLOSED_AMT_CREDIT_SUM_DEBT_MAX": 0.0,
            "CLOSED_AMT_CREDIT_SUM_LIMIT_MEAN": 0.0,
            "CLOSED_AMT_CREDIT_SUM_LIMIT_MAX": 0.0,
            "CLOSED_AMT_CREDIT_SUM_OVERDUE_MEAN": 0.0,
            "CLOSED_AMT_CREDIT_SUM_OVERDUE_MAX": 0.0,
            "CLOSED_DAYS_CREDIT_UPDATE_MEAN": -661.3333333333,
            "CLOSED_DAYS_CREDIT_UPDATE_MAX": -34.0,
            "CLOSED_AMT_ANNUITY_MEAN": 0.0,
            "CLOSED_AMT_ANNUITY_MAX": 0.0,
            "CLOSED_FLAG_MISSING_AMT_ANNUITY_MEAN": 0.1666666667,
            "CLOSED_FLAG_MISSING_AMT_ANNUITY_MAX": 1.0,
            "CLOSED_FLAG_MISSING_CREDIT_MAX_OVERDUE_MEAN": 0.3333333333,
            "CLOSED_FLAG_MISSING_CREDIT_MAX_OVERDUE_MAX": 1.0,
            "CLOSED_CREDIT_INFO_MISSING_RATIO_MEAN": 0.3333333333,
            "CLOSED_CREDIT_INFO_MISSING_RATIO_MAX": 0.5,
            "CLOSED_AMT_CREDIT_DEBT_RATE_MEAN": 0.0,
            "CLOSED_AMT_CREDIT_DEBT_RATE_MAX": 0.0,
            "CLOSED_FLAG_ACTIVE_CREDIT_MEAN": 0.0,
            "CLOSED_FLAG_ACTIVE_CREDIT_MAX": 0.0,
            "CLOSED_MONTHS_BALANCE_MIN_MIN": -47.0,
            "CLOSED_MONTHS_BALANCE_MAX_MAX": 0.0,
            "CLOSED_MONTHS_BALANCE_SIZE_MEAN": 15.0,
            "CLOSED_MONTHS_BALANCE_SIZE_MAX": 22.0,
            "PREV__TOTAL_MISSING_RATIO": 0.0140186916,
            "APPROVED__TOTAL_MISSING_RATIO": 0.0434782609,
            "INSTAL__TOTAL_MISSING_RATIO": 0.0,
            "CC__TOTAL_MISSING_RATIO": 1.0,
            "POS__TOTAL_MISSING_RATIO": 0.0,
            "BURO__TOTAL_MISSING_RATIO": 0.0,
            "ACTIVE__TOTAL_MISSING_RATIO": 0.0512820513,
            "CLOSED__TOTAL_MISSING_RATIO": 0.0,
        }
    ]
)